## Programming for Data Analysis - Project 2 ##

**Name: James McEneaney** <br/><br/>
**Course: Higher Diploma in Computing in Data Analytics, ATU Ireland** <br/><br/> 
**Semester: Semester 2 2023** <br/><br/>


***

### Introduction ###

This project is the second of two projects I have completed for the "Programming for Data Analysis" module of the Higher Diploma in Data Analytics in ATU Ireland.

The aim of the project is to analyse various aspects of climate data from the Paleolithic (oldest dataset goes back to 820,000 years ago) up to the present day. I will make predictions for future climate data based on recent trends.

This project will begin with the importing of libraries used to analyse the data. I will then load up the datasets and amend them to prepare them for analysis. The data-cleansing section will also describe the data which is contained in each dataset and plot some of them to provide a sense of how the data in each is distributed.

I will then compared the datasets relating to CO2 levels to each other and amalgamate them, before plotted them against the data for the Antarctic temperature anomaly.

Next, I will plot data for CH4 (Methane) against the Antarctic temperature anomaly and analyse the relationship.

I will then plot CO2 and CH4 against each other over the full 820,000 timespan of data.

Next I will investigate climate change signals within an Irish context by analysing the trends for temperature and rainfall at Mullingar weather station; I will measure these by calculating anomalies for each year in the dataset relative to a base period. Then, I will analyse the temperature anomaly against the Global temperature anomaly, and the temperature and rainfall anomalies against each other.

When I complete this data analysis, I will fuse my datasets for CO2, CH4, temperature and rainfall into one composite dataframe, before exporting it to csv and json files; these are found in this repository under the titles "composite climate data.csv" and "composite climate data.json" respectively.

After doing this, I will use a linear regression model to predict how the global temperature anomaly would unfold between now and 2100, under the assumption that the trend in this anomaly since 1950 would continue in a linear fashion; I will compare my findings witrh the published findings in the "Summary for Policymakers" document written by climate experts for the Intergovernmental Panel on Climate change.

My project will conclude with a summary of my findings and thoughts on completion of the project, followed by a list of references I used to complete my work.

### Table of Contents ###
* [Introduction](#introduction)
* [Data-cleansing](#data-cleansing)
    * [Description of datasets](#description-of-datasets)
    * [Loading up datasets](#loading-up-datasets)
    * [Plots of datasets](#plots-of-datasets)
    * [Comparison of CO2 datasets](#comparison-of-co2-datasets)
    * [Combining IPCC and Mauna Loa CO2 datasets](#combining-ipcc-and-mauna-loa-co2-datasets)
* [CO2 versus Temperature Anomaly](#co2-versus-temperature-anomaly)
    * [Preparing Temperature Anomaly dataset](#preparing-temperature-anomaly-dataset)
    * [Plotting CO2 versus Temperature anomaly](#plotting-co2-versus-temperature-anomaly)
* [CH4 versus Temperature Anomaly](#ch4-versus-temperature-anomaly)
    * [Plotting CH4 versus Temperature Anomaly](#plotting-ch4-versus-temperature-anomaly)
    * [Correlation](#correlation)
* [CH4 versus CO2](#ch4-versus-co2)
    * [Plotting CH4 versus CO2](#plotting-ch4-versus-co2)
    * [Correlation and lead-lag relationship](#correlation-and-lead-lag-relationship)
* [Climate change signals in the Irish context](#climate-change-signals-in-the-irish-context)
    * [Background to Irish climate analysis](#background-to-irish-climate-analysis)
    * [Plotting Mullingar climate anomalies](#plotting-mullingar-climate-anomalies)
    * [Mullingar temperature anomaly versus Global temperature anomaly](#mullingar-temperature-anomaly-versus-global-temperature-anomaly)
    * [Mullingar temperature anomaly versus rainfall anomaly](#mullingar-temperature-anomaly-versus-rainfall-anomaly)
* [Fusion of datasets](#fusion-of-datasets)
* [Prediction of Global Temperature Anomaly](#prediction-of-global-temperature-anomaly)
* [Accelerating increases of temperature](#accelerating-increases-of-temperature)
* [Summary](#summary)
* [References](#references)

### Data-cleansing ###

#### Description of datasets ####

To begin, I will download the libraries I will use in this project. I will use pandas to create the dataframes which I will use to analyse the historical climate data:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression 

I have downloaded multiple climate related datasets online. These datasets can be found in the subfolder 'climate_data' in this repository; they are as follows:

- Atmospheric **carbon dioxide** concentratons from a report by the Intergovernmental Panel on Climate Change (IPCC); most recent year in dataset is 2001.

- Atmospheric **carbon dioxide** concentrations from a 2008 paper in Nature by Luthi et al; the most recent year in this dataset is 1870 (137 years prior to 2007).

- Atmospheric **carbon dioxide** data from the Mauna Loa Observatory in Hawaii, providing annual data from 1959 to 2022, when precise measurements of atmospheric carbon dioxide first began to be taken. Previously, the analyser tool worked using infrared absorption, but since 2019 it has worked on the basis of a technique called 'Cavity Ring-Down Spectroscopy'. Measurements are assured to be reflective of the true composition of the global atmosophere, due to the location of the observatory and the rigour with which measurements are calibrated and independently verified.

- **Temperature** estimates going back over 820,000 years based on the EPICA Dome C Ice Core Deuterium Data. EPICA stands for European Project for Ice Coring in Antarctica. It estimates past temperature using deuterium as a proxy. Deuterium is a heavy form of hydrogen (containing one neutron in addition to the one proton found in "light" hydrogen). There tends to be less deuterium in ice cores associated with cooler periods than in warmer periods. The reason for this is that during warm periods, more of the lighter form of hydrogen is evaporated away from the surface of the ocean, so that when the moist air moves northwards to the poles, it contains a higher ratio of heavy hydrogen than it contains during cold periods; precipitation then contains more of this heavy hydrogen and this ultimately shows up in the ice cores. I will refer to this dataset as the "Jouzel" dataset for convenience (Jean Jouzel is a leading glaciologist and climatologist who is one of the creators of the dataset). I will use this dataset to calculate the Anarctic temperature anomaly for the period from 820,000 years ago to 1850 AD.

- Global **temperature anomaly** data from the National Oceanic and Atmospheric Administration (NOAA). Measurements for the anomaly are taken from 1850 to 2022 and are calculated against the average temperature for the period 1901 to 2000 and are measured in °C Celsius.

- Antarctic **temperature anomaly** data from the National Oceanic and Atmospheric Administration (NOAA). Measurements for the anomaly are taken from 1850 to 2022 and are calculated against the average temperature for the period 1910 to 2000 and are measured in °C Celsius.

- Atmospheric **methane** concentrations estimated from ice core data from Antarctica EPICA Dome C and Law Dome. This covers the period from 797446 years ago to the year 1980. The record until around 1004 AD relates entirely to Dome C; the record between around 1004 AD and 1824 AD relates to a combination of Dome C and Law Dome, while the record from 1824 onwards relates almost entirely to Law Dome. I cut the relevant data from a source csv file from the Environmental Protection Agency website and saved it into my working directory; I merged and manipulated the data further using python. Figures for methane are measured in parts per billion by volume.

- Recent global atmospheric **methane** concentrations obtained from the NOAA website. Atmospheric methane concentrations are determined from marine surface sites. This dataset covers the period 1984 to 2022.

Later on I will upload datasets relating to Mullingar weather station which I will use to analyse climate trends in the Irish context; I will describe these datasets in the section on 'climate change signals in the Irish context'. 

I will also uploaded a dataset of monthly data for the Global temperature anomaly for the period January 2018 to November 2023 which I will use to investigate the possibility of accelerated warming in the recent past.

#### Loading up datasets ####

With the exception of the Mauna Loa dataset, these datasets were not in csv format initially. To make it easier to create dataframes using pandas, I saved all my source files as csv files in my working directory. I will edit the dataframes using python before I analyse the data; this will include renaming column headings to make them easier to work with and I will add extra columns to datasets to standardise the figures for time measurement and have the oldest years listed first, so my plots print from oldest years at the left to most recent years at the right.

To begin, I will load up and amend the IPCC CO2 dataset:

In [ ]:
df_co2_ipcc = pd.read_csv("climate_data\CO2_ipcc_csv.csv")

# rename column names of existing dataframe (setting 'inplace' parameter equal to 'True')
df_co2_ipcc.rename(columns={'Gasage (yr BP) ': 'Year before 1950', 'CO2 (ppmv)': 'CO2', 'sigma mean CO2 (ppmv)': 'sigma mean CO2'}, inplace=True) 

# here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# I am adding 73 years to every value in the 'years before 1950' column to effectively convert the column to 'years before 2023'. 
# I am then multiplying each year by minus 1, so that the column represents years "in the past"
df_co2_ipcc['Years'] = (df_co2_ipcc['Year before 1950'] + 73) * -1

# reversing the order of the dataset, to for the older years to appear on the left-hand side of the x-axis
df_co2_ipcc = df_co2_ipcc[::-1]

print(df_co2_ipcc)

Next, I will load up and amend the 2008 Nature paper dataset:

In [ ]:
df_co2_nature = pd.read_csv("climate_data/CO2_nature_csv.csv")

df_co2_nature.rename(columns={'EDC3_gas_a (yr)': 'Years ago', 'CO2 (ppmv)': 'CO2'}, inplace=True)

# Since the paper was written in 2007 and the most recent year in the associated dataset is 137 year prior, I have added 16
# years to the figure for "years ago" when creating the new column 'Years'
df_co2_nature['Years'] = (df_co2_nature['Years ago'] + 16) * -1

df_co2_nature = df_co2_nature[::-1]

print(df_co2_nature)

Next I will load up the relatively small dataset from Mauna Loa, containing data for atmospheric CO2 levels:

In [ ]:
df_mauna_loa_csv = pd.read_csv("climate_data/mauna_loa_csv.csv")

# I will rename the columns to standardise the heading for the CO2 data:
df_mauna_loa_csv.rename(columns={'year': 'year', 'mean': 'CO2', 'unc':'unc'}, inplace=True)

# I will create a new column called 'Years', to standardise the measurement of time across the datasets.
# I don't need to reverse the rows in this dataset since the earliest year is already in the first row.
df_mauna_loa_csv['Years'] = (2023 - df_mauna_loa_csv['year']) * -1

print(df_mauna_loa_csv)

Next, the "Jouzel" temperature data inferred from deuterium levels in ice cores from EPICA Dome C:

In [ ]:
df_temp_jouzel_csv = pd.read_csv("climate_data/temp_jouzel_csv.csv")

df_temp_jouzel_csv.rename(columns={'bag': 'bag', 'ztop': 'depth', 'EDC3béta': 'Years before 1950', 'AICC2012': 'year_new', 'deutfinal': 'deuterium', 'temp': 'temp_Kelvin', 'acc-EDC3beta' : 'acc-EDC3beta'}, inplace=True)

# Adding a new column called 'Years' to standardise the measurement of time across datasets. 
# This represents the number of years ago the data relates to eg. -55 relates to (-55 + 73 = 18 years ago, or the year 2005)
# The negative figure allows for my graph to print from oldest date to most recent
df_temp_jouzel_csv['Years'] = (df_temp_jouzel_csv['year_new'] + 73) * -1

# adding a new column called 'temp_Celsius' as I am more familiar with this unit of measurement for temperature and would
# like to plot the data in terms of Celsius 
df_temp_jouzel_csv['temp_Celsius'] = (df_temp_jouzel_csv['temp_Kelvin'] - 273.15)

df_temp_jouzel_csv = df_temp_jouzel_csv[::-1]

print(df_temp_jouzel_csv)

I will now load up the dataset for the Antarctic temperature anomaly and create a new column which contains standardised data for the year to allow for easier comparison with other datasets:

In [ ]:
df_antarctic_temp_anomaly_1850 = pd.read_csv("climate_data/antarctic_temp_anomaly_csv.csv")

# Renaming columns to standardise across datasets when combining later on
df_antarctic_temp_anomaly_1850.rename(columns={'Year': 'year', 'Anomaly': 'anomaly'}, inplace=True)

# Cdding a new column called 'Years' to standardise the measurement of time across datasets. 
# Rhis represents the number of years ago the data relates to eg. - 1 is 1 year ago ie. 2022
df_antarctic_temp_anomaly_1850['Years'] = (df_antarctic_temp_anomaly_1850['year'] - 2023)

print(df_antarctic_temp_anomaly_1850)

I will also upload the data for Global temperature anomoly from 1850 to 2022, and add a standardised column for 'Years' as I did for the Antarctic temperature anomaly data:

In [ ]:
df_global_temp_anomaly_1850 = pd.read_csv("climate_data/global_temp_anomaly_csv.csv")

# renaming columns to standardise across datasets when combining later on
df_global_temp_anomaly_1850.rename(columns={'Year': 'year', 'Anomaly': 'anomaly'}, inplace=True)

# adding a new column called 'Years' to standardise the measurement of time across datasets. 
# this represents the number of years ago the data relates to eg. - 1 is 1 year ago ie. 2022
df_global_temp_anomaly_1850['Years'] = (df_global_temp_anomaly_1850['year'] - 2023)

print(df_global_temp_anomaly_1850)

I will now create dataframes for the data relating to **methane** and modify them where necessary. Firstly, the paleo data:

In [ ]:
df_ch4_paleo = pd.read_csv("climate_data/ch4_paleo_csv.csv")

# Here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# The original dataset split the years by AD/BC so I subtracted 2023 from each year to standardise the measurement of time.
df_ch4_paleo['Years'] = (df_ch4_paleo['year'] - 2023)

# I am converting the figures in the 'CH4' column to float to avoid getting warnings later on if I try to merge
# integers with floats
df_ch4_paleo['Years'] = df_ch4_paleo['Years'].astype(float)

# Here I am merging two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column. 
# This is to create just one column of paleo data for atmospheric methane concentration.
df_ch4_paleo['CH4'] = df_ch4_paleo['dome_c'].combine_first(df_ch4_paleo['law_dome'])

# Next I am dropping the unwanted columns 'dome_c', 'law_dome' and 'year'. This gives us
# a dataframe containing only years and atmospheric CH4 concentrations up until 1980.
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4_paleo = df_ch4_paleo.drop(['dome_c', 'law_dome', 'year'], axis = 1)

print(df_ch4_paleo)

Secondly, the more recent methane data:

In [ ]:
df_ch4_recent = pd.read_csv("climate_data/ch4_recent_csv.csv")

# Here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# The original dataset split the years by AD/BC so I subtracted 2023 from each year to standardise the measurement of time.
df_ch4_recent['Years'] = (df_ch4_recent['year'] - 2023)

# I am converting the figures in the 'Years' columns to float to avoid getting a warning message later when I try to 
# combine the 'Years' data from the recent data (which is in integer format) with the 'Years' data from the 
# paleo dataset, which is in float format
df_ch4_recent['Years'] = df_ch4_recent['Years'].astype(float)

# Next I am dropping the unwanted column 'year'. This gives us a dataframe containing only years 
# and atmospheric CH4 concentrations from 1984 to 2022.
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4_recent = df_ch4_recent.drop(['year'], axis = 1)

print(df_ch4_recent)

Finally, we can merge the paleo and recent methane datasets to enable us to plot methane levels later on:

In [ ]:
df_ch4 = pd.merge(df_ch4_paleo, df_ch4_recent, on = 'Years', how = 'outer')

# The merge produces two columns: 'CH4_x' relating to the paleo data and 'CH4_y' relating to the recent data.
# Here I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_ch4['CH4'] = df_ch4['CH4_x'].combine_first(df_ch4['CH4_y'])

# Here I am dropping the unwanted columns CH4_x and CH4_y. 
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4 = df_ch4.drop(['CH4_x', 'CH4_y'], axis = 1)

print(df_ch4)

#### Plots of datasets ####

Below are plots of the CO2 and temperature datasets which I wanted to plot to familiarise myself with them. I have included grids on plots where I felt they would enhance their readibility:

*IPCC data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.xlabel('Years')
plt.ylabel('CO2 (ppmv)')
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years up to 2001 AD - IPCC data')

sns.scatterplot(x ='Years', y = 'CO2', data = df_co2_ipcc, size = 20)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x ='Years', y = 'CO2', data = df_co2_ipcc, color = 'blue')

plt.savefig("images/IPPC_CO2.png")

*Nature data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.xlabel('Years')
plt.ylabel('CO2 (ppmv)')
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years up to 1870 AD - Nature data')

sns.scatterplot(x='Years', y ='CO2', data=df_co2_nature, color = 'orange', size = 20)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x = 'Years', y = 'CO2', data = df_co2_nature, color = 'orange')

plt.savefig("images/Nature_CO2.png")

*Mauna Loa:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) since 1959 - Mauna Loa data')
plt.xlabel('Year')
plt.ylabel('CO2 (ppmv)')
plt.grid()

# Here I will plot using the column 'year' as opposed to 'Years', as the graph for recent data is easier to comprehend 
# when the x-axis is labelled by calendar year rather than how many years ago the year was
# We use the 'size' parameter to decrease size of markers 
sns.scatterplot(x = 'year', y = 'CO2', data=df_mauna_loa_csv, color = 'brown', size = 20)      

sns.lineplot(x = 'year', y = 'CO2', data = df_mauna_loa_csv, color = 'brown')

plt.savefig("images/Mauna_Loa_CO2.png")

*Jouzel data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Antarctic air temperature estimates in °C for past 820000 years inferred from deuterium measurements in EPICA Dome C ice cores')
plt.xlabel('Years')
plt.ylabel('Temperature (°C)')
plt.grid(True)   

sns.scatterplot(x = 'Years', y = 'temp_Celsius', data=df_temp_jouzel_csv, color = 'green', size = 20)      

sns.lineplot(x = 'Years', y = 'temp_Celsius', data = df_temp_jouzel_csv, color = 'green')

plt.savefig("images/Jouzel_Antarctic_temp.png")

*Antarctic temperature anomaly 1850 to 2022:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Antarctic temperature anomaly (vs 1910 - 2000 average) in °C for the period 1850 to 2022')
plt.xlabel('Years')
plt.ylabel('Temperature Anomaly (°C)')
plt.grid(True)

# Here I will plot using the column 'year' as opposed to 'Years', as the graph for recent data is easier to comprehend 
# when the x-axis is labelled by calendar year rather than how many years ago the year was
# We use the 'size' parameter to decrease size of markers 
sns.scatterplot(x = 'year', y = 'anomaly', data=df_antarctic_temp_anomaly_1850, color = 'red', size = 20)      

sns.lineplot(x = 'year', y = 'anomaly', data = df_antarctic_temp_anomaly_1850, color = 'red')

plt.savefig("images/Antarctic_temp_anomaly.png")

*Global temperature anomaly:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Global temperature anomaly (vs 1901 to 2000 average) in °C for the period 1850 to 2022')
plt.xlabel('Years')
plt.ylabel('Temperature Anomaly (°C)')
plt.grid(True)

# Here I will plot using the column 'year' as opposed to 'Years', as the graph for recent data is easier to comprehend 
# when the x-axis is labelled by calendar year rather than how many years ago the year was
# We use the 'size' parameter to decrease size of markers 
sns.scatterplot(x = 'year', y = 'anomaly', data=df_global_temp_anomaly_1850, color = 'red', size = 20)      

sns.lineplot(x = 'year', y = 'anomaly', data = df_global_temp_anomaly_1850, color = 'red')

plt.savefig("images/Global_temp_anomaly.png")

*Methane (composite dataset):*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric Methane concentrations (parts per billion by volume) from 800000 year ago to 2022')
plt.xlabel('Years')
plt.ylabel('Methane concentration (ppbv)')
plt.grid(True)

# Here I will plot using the column 'year' as opposed to 'Years', as the graph for recent data is easier to comprehend 
# when the x-axis is labelled by calendar year rather than how many years ago the year was
# We use the 'size' parameter to decrease size of markers 
sns.scatterplot(x = 'Years', y = 'CH4', data=df_ch4, color = 'purple', size = 20)      

sns.lineplot(x = 'Years', y = 'CH4', data = df_ch4, color = 'purple')

plt.savefig("images/Methane_composite.png")

#### Comparison of CO2 datasets ####

To get a sense of the extent to which carbon dioxide levels may have increased in recent years compared to the long-term historical trends, I would ideally have a dataset which contains carbon-dioxide data from the distant past right up to the present day. 

Fortunately, I can obtain this by combining the data from the IPCC dataset with the more recent data from the Mauna Loa dataset. 

Firstly, I want give myself some confidence that the long-term historical data in the IPCC dataset is valid; to do this, I will compare it with the 2008 Nature dataset by creating plot a lineplot for each dataset on the same graph:

In [ ]:
plt.figure(figsize= (16,6))

sns.lineplot(x='Years', y='CO2', data=df_co2_ipcc, color = 'blue', label='IPCC')
sns.lineplot(x='Years', y='CO2', data=df_co2_nature, color = 'orange', label='Nature')

plt.title('Atmospheric CO2 over 800000 years - IPCC vs Nature data')
plt.xlabel('Years')
plt.ylabel('CO2 (ppmv)')

plt.legend()

plt.savefig('images/CO2_IPCC_vs_Nature.png')

plt.show()

We see that the datasets are largely in agreement with one another. The Nature data ends in 1870, while the IPCC data continues on to 2001, and the large spike in carbon dioxide levels seen on the right hand-side of the graph relates to these additional years present in the IPCC data.

We can also plot the most recent carbon dioxide data from the IPCC dataset with it's counterpart data from the Mauna Loa dataset:

In [ ]:
# create a new sub-dataframe from the IPCC dataframe containing only rows relating to 1959 onwards 
#(ie. up to last data point of 2001)
df_co2_ipcc_1959_to_2001 = df_co2_ipcc.loc[df_co2_ipcc['Years'] >= -64]

# create a new sub-dataframe from the Mauna Loa dataframe containing only rows relating to 2001 backwards 
# (ie. back to first data point of 1959)
df_mauna_loa_1959_to_2001 = df_mauna_loa_csv.loc[df_mauna_loa_csv['Years'] <= -22]

#print(df_co2_ipcc_1959_to_2001)

#print(df_mauna_loa_1959_to_2001)

Now we can plot the data from these two sub-dataframes against each other:

In [ ]:
plt.figure(figsize= (16,6))

sns.lineplot(x='Years', y='CO2', data=df_co2_ipcc_1959_to_2001, color = 'blue', label='IPCC')
sns.lineplot(x='Years', y='CO2', data=df_mauna_loa_1959_to_2001, color = 'brown', label='Mauna Loa')

plt.title('Atmospheric CO2 between 1959 and 2001 - IPCC vs Mauna Loa data')
plt.xlabel('Years')
plt.ylabel('CO2 (ppmv)')

plt.legend()

plt.savefig('images/CO2_IPCC_vs_Mauna_Loa.png')

plt.show()

It should be noted that the IPCC data relates to CO2 levels inferred from high resolution ice cores in the Law Dome in East Antarctica while the Mauna Loa data comes from the summit of Mauna Loa in Hawaii, using a sophisticated analyser tool. Nevertheless, we can see that the data for CO2 concentration during the overlapping period of the IPCC and Mauna Loa datasets is in agreement. Comparison of the carbon dioxide datasets which each other gives us confidence that the composite dataset is a reliable record of atmospheric carbon dioxide levels going back over 800,000 years.

#### Combining IPCC and Mauna Loa CO2 datasets ####

I will now combine the IPCC data with the Mauna Loa data to create a new dataframe containing data for carbon dioxide levels from 805,741 years ago to today:

In [ ]:
# Firstly, I will create a sub-dataframe from the Mauna Loa containing CO2 data relating to 2002 onwards
df_post_2001_mauna_loa = df_mauna_loa_csv.loc[df_mauna_loa_csv['Years'] >= -21 ]

# Creating a new dataframe from the Mauna Loa data, containing only the two columns of interest 'CO2' and 'Years'.
# To avoid getting a warning for 'chained indexing', I followed the syntax as found in this link : 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
df_post_2001_mauna_loa_trimmed = df_post_2001_mauna_loa.loc[:, ('Years', 'CO2')]

# I am converting the figures in the years columns to float to avoid getting a warning message when I combine the 'Years' data
# from the Mauna Loa data (which is in integer format) with the 'Years' data from the IPCC dataset, which is in float format
df_post_2001_mauna_loa_trimmed['Years'] = df_post_2001_mauna_loa_trimmed['Years'].astype(float)

#print(df_post_2001_mauna_loa_trimmed)

# creating a new dataframe from the IPCC data, containing only the two columns of interest 'CO2' and 'Years'
df_co2_ipcc_trimmed = df_co2_ipcc.loc[:, ('Years', 'CO2')]

#print(df_co2_ipcc_trimmed)

# Here I am merging the trimmed dataframes for the IPCC data and post-2001 Mauna Loa data 
df_co2 = pd.merge(df_co2_ipcc_trimmed, df_post_2001_mauna_loa_trimmed, on = 'Years', how = 'outer')

# The merge produces two columns: 'C02_x' relating to the IPCC data and 'CO2_y' relating to the Manua Loa data.
# Here I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_co2['CO2'] = df_co2['CO2_x'].combine_first(df_co2['CO2_y'])

# Here I am dropping the unwanted columns CO2_x and CO2_y. the axis = 1 parameter is used to specify that it is a column being
# dropped as opposed to a row
df_co2 = df_co2.drop(['CO2_x', 'CO2_y'], axis = 1)

print(df_co2)

Next, I will plot this composite carbon dioxide dataset to show how the most recent figures for carbon dioxide levels compare with the long-term historical average:

In [ ]:
plt.figure(figsize= (16,6))
plt.xlabel('Years')
plt.ylabel('CO2 (ppmv)')
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years')

sns.scatterplot(x ='Years', y ='CO2', data=df_co2, color ='blue', size = 20)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x = 'Years', y = 'CO2', data = df_co2, color = 'blue')

plt.savefig('images/CO2_composite.png')

We can see clearly that atmospheric carbon dioxide levels have risen greatly in a very short period of time to reach levels significantly higher than the levels which prevailed for the past 800000 years; the level for 2022 was 418.53 ppm, which is much greater than range of roughly 180 to 280 ppm which seems to have been the level for most of the time covered here.

### CO2 versus Temperature Anomaly ###

#### Preparing Temperature Anomaly dataset ####

To compare CO2 levels for the past 820000 years with the temperature anomaly, I will need to obtain a dataset for the temperature anomaly covering this whole timespan. We can use data from the NOAA for the Antarctic air temperature anomaly for the period from 1850 to now. This record compares the temperature for each year to the average temperature of the base period 1910 to 2000. 

**Note**: I will refer to this dataset as the "temperature anomaly" dataset but in effect it can be considered the "Antarctic temperature anomaly" dataset; the "Global temperature anomaly" dataset from the NOAA only contains data going back to only 1850, so I decided to use the Antarctic NOAA data instead to bring my Jouzel data to the present day, since the Jouzel dataset contains temperature anomaly data relating to Antarctica.

For earlier data, I will calculate the temperature anomaly as follows:
I will take the average temperature for the period 1000 AD to 1845 AD from the Jouzel dataset, calculated based on the number of data points for temperature during this timespan; I am choosing 1845 AD since this is the most recent figure for temperature given in the Jouzel dataset which doesn't have a counterpart datapoint in the NOAA dataset. The reason I am using almost 845 years of data as a base period is that a similar base period (the last 1000 years) was used to calculate the temperature anomaly in the earlier Jouzel dataset (found on the text file here: https://www1.ncdc.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt). I wasn't able to easily extract this temperature anomaly data from the text file so I decided to calculate it myself based on the newer dataset (saved in my working directory as a csv file titled **'temp_jouzel_csv.csv'**).

Since 1845 is a relatively early date in the timeline of humanity's history of causing global warming, it seems acceptable to me to compare the temperature in 1845 to the pre-industrial temperatures of earlier times. 

I will then create a dataframe containing the temperature anomaly going back to 820000 years ago, up to the present day. I will plot these figures against the figures for CO2 obtained above.

In [ ]:
# First I will create a sub-dataframe from the Jouzel dataset containing temperature data relating to 1845 backwards
df_pre_1845_jouzel = df_temp_jouzel_csv.loc[df_temp_jouzel_csv['Years'] <= -178 ]

# Creating a new dataframe from the Jouzel sub-dataframe, containing only the two columns of interest 'Years' and 'temp_Kelvin'
# To avoid getting a warning for 'chained indexing', I followed the syntax as found in this link : 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
df_pre_1845_jouzel_trimmed = df_pre_1845_jouzel.loc[:, ('Years', 'temp_Kelvin')]

# I am converting the figures in the years columns to float to avoid getting a warning messages if I try to combine 
# floats and integers when combining 'years' data from different datasets
df_pre_1845_jouzel_trimmed['Years'] = df_pre_1845_jouzel_trimmed['Years'].astype(float)

# to calculate the average temperature between 1000 AD and 1845 AD, we only take rows relating to 1023 years ago and onwards
df_1000_to_1845_jouzel = df_pre_1845_jouzel_trimmed.loc[df_pre_1845_jouzel_trimmed['Years'] >= - 1012]

mean_temp_1000_to_1845 = df_1000_to_1845_jouzel['temp_Kelvin'].mean()
round_mean_temp_1000_to_1845 = round(mean_temp_1000_to_1845, 2)
print(f"The mean Antarctic air temperature between 1000 AD and 1845 AD is estimated to be {round_mean_temp_1000_to_1845} Kelvin \n")

# Next I want to calculate the temperature anomaly for each year in the Jouzel dataset using this base period temperature of 217.18 kelvin:
df_pre_1845_jouzel_trimmed['anomaly'] = df_pre_1845_jouzel_trimmed['temp_Kelvin'] - 217.16
print("The temperature anomaly in degrees compared with the average for the period 1000 to 1845 is as follows: \n ")
# print(df_pre_1845_jouzel_trimmed)

# Next I want to combine this pre 1845 temperature anomaly data with the post 1850 data from the NOAA:
# Firstly I will ensure the data for 'Years' in the NOAA daaset is in float format, to avoid getting a warning
# for trying to merge floats and ints in the same column
df_antarctic_temp_anomaly_1850['Years'] = df_antarctic_temp_anomaly_1850['Years'].astype(float)

df_temp_anomaly = pd.merge(df_pre_1845_jouzel_trimmed, df_antarctic_temp_anomaly_1850, on = 'Years', how = 'outer')

# The merge produces two anomaly columns: 'anomaly_x' relating to the Jouzel data and 'anomaly_y' relating to the NOAA data.
# Below I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_temp_anomaly['anomaly'] = df_temp_anomaly['anomaly_x'].combine_first(df_temp_anomaly['anomaly_y'])

# Next I am dropping the unwanted columns 'anomaly_x', 'anomaly_y', 'year' and 'temp_Kelvin'. This gives us
# a dataframe containing only years and the temperature anomly for each year in degrees (Celsius or Kelvin).
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_temp_anomaly = df_temp_anomaly.drop(['anomaly_x', 'anomaly_y', 'year', 'temp_Kelvin'], axis = 1)
                                    
print(df_temp_anomaly)

#### Plotting CO2 versus Temperature anomaly ####

We can now plot the composite dataset for CO2 levels against the composite dataset for the temperature anomoly, over approximately 820000 years:

In [ ]:
# Here I created a graph with two y-axes but with each y-axis' data measured against the same x-axis
# https://python-graph-gallery.com/line-chart-dual-y-axis-with-matplotlib/#:~:text=It%20uses%20ax.,Y%20axis%20on%20this%20twin.

fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CO2 on the primary y-axis
ax1.plot(df_co2['Years'], df_co2['CO2'], color='blue', label='CO2')
ax1.set_xlabel('Years')
ax1.set_ylabel('CO2 (ppmv)', color='blue')
ax1.tick_params(axis='y', colors='blue')   

# Plot for temperature on secondary y-axis
ax2.plot(df_temp_anomaly['Years'], df_temp_anomaly['anomaly'], color='red', label='Temperature anomaly')
ax2.set_ylabel('Temperature Anomaly (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('CO2 versus Antarctic temperature anomaly over the past 820000 years') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CO2_vs_temp_anomaly_820000_ya.png')
plt.show()

It seems to be the case that, over the past 820000 years, the temperature anomoly was associated with the level of CO2 in the atmosphere, since local peaks in the data for CO2 tend to occur during the same periods as local peaks in the temperature anomaly data. 

We can also see that, for most of the past 820000 years, the average temperature at Antarctica was colder than it is at present (most of the temperature anomaly plot is beneath 0 °C on the right-hand y-axis); this is to be expected, since for most of that period the earth was in a glacial period, with relatively warm interglacial periods like the past 10000 years being the exception.

To compare the data for the Antarctic temperature anomaly from 1850 with the figures for CO2 from 1850, I can create a sub-dataframe for the CO2 for 1850 to 2022 and plot it in the same way I plotted the full datasets; by doing this, we can investigate the temperature anomaly as it relates to recent increases in CO2 levels in the atmosphere. It should be borne in mind that the figures for temperature anomaly for 1850 onwards are calculated based on a the average Antarctic air temperature for the period 1910 to 2000.

In [ ]:
df_co2_post_1850 = df_co2.loc[df_co2['Years'] >= -173]
print(df_co2_post_1850)

# I will also create a sub-dataframe from the composite temperature anomaly dataset to include only years from 1850 onwards,
# as I want to use the 'Years' column which has a standardised format for measuring time
df_temp_anomaly_post_1850 = df_temp_anomaly.loc[df_temp_anomaly['Years'] >= -173]
print(df_temp_anomaly_post_1850)

We can now plot this CO2 data against the temperature anomaly data for the "industrial" period from 1850 to 2022:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CO2 on the primary y-axis
ax1.plot(df_co2_post_1850['Years'], df_co2_post_1850['CO2'], color='blue', label='CO2')
ax1.set_xlabel('Years')
ax1.set_ylabel('CO2 (ppmv)', color='blue')
ax1.tick_params(axis='y', colors='blue')   

# Plot for temperature anomaly on secondary y-axis. 
ax2.plot(df_temp_anomaly_post_1850['Years'], df_antarctic_temp_anomaly_1850['anomaly'], color='red', label='Temperature')
ax2.set_ylabel('Temperature Anomaly (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('Antarctic Atmospheric CO2 levels versus Antarctic temperature anomaly from 1850 to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CO2_vs_temp_anomaly_post_1850.png')
plt.show()

When we plot these two variables together, we see that the data can be divided into two sections:

- The data from 1850 to 1950 shows that temperature didn't fluctuate greatly and was typically lower than the average for 1910 to 2000 as a whole (is beneath 0°C on the right-hand y-axis). Carbon dioxide levels also didn't increase hugely over this time period relative to the increases which came after that point (levels increased from around 286 ppm to 313 ppm between 1850 and 1950 according to the IPCC data: an increase of around 9.4% over a century)

- The data from 1950 to 2022 shows that temperatures fluctuated a lot more compared to the average for 1910 to 2000 and that for most of the period since around 1965, average temperature at the Antarctic has been higher than this base figure. One interesting point I notice is that the temperature anomaly reached levels of over 0.5°C above baseline during four years from 1988 to 2007, but only surpassed 0.4°C above baseline twice since 2007; an interesting potential investigation might be to compare levels of CFCs released into the atmosphere versus the temperature anomaly, as the use of CFCs began to be banned from the late 1980s onwards when it was recognised that they played a huge role destroying the ozone layer and in causing global warming. 

A manual investigation of the IPCC dataset shows that it took around **53 years** (from 1912 to 1965) for CO2 levels to go from 300 to 320 ppm. According to the Mauna Loa dataset, it then took only **16 years** to increase to 340 ppm (in 1981), **14 years** to reach 360 ppm (in 1995), **10 years** to reach 380 ppm (in 2005) and **10 years** to reach 400 ppm in 2015. The most recent figures for the year 2022 are 418 ppm.

The figures leave no doubt as to the enormous and rapid impact human activity is having on our atmosphere and climate.

#### Correlation and lead-lag analysis: ####

I now want to test for correlation between CO2 and the Antarctic temperature anomaly, but to do so I need the data from each dataset to align. This is very difficult to obtain for the longer-term datasets in this case, because there are 1922 rows in the CO2 dataset and 5971 rows in the temperature anomaly dataset, and taking a sample of values from the temperature anomaly data and matching them to the CO2 would introduce a lot of randomness into the final figure for correlation. In any case, the 'years' for the CO2 data set contain decimal places while the 'years' for the temperature anomaly data don't, so they would not align even if the datsets were the same size or closer in size.

Instead, I will test out lead-lag relationships between CO2 and the Antarctic temperature anomaly for the period 1959 to 2022 using the Mauna Loa CO2 data set: this dataset begins in 1959 and the figures for 'years' do not contain decimal places, so can be easily aligned with the the 'years' column for the temperature anomaly data going back to 1959.

Firstly, I will create a sub-dataframe from the temperature anomaly data containing data relating to the period 1959 to 2022. I will then calculate the correlation coefficient between the anomalies in that sub-dataframe and the figures for CO2 in the Mauna Loa dataset:

In [ ]:
df_temp_anomaly_post_1959 = df_temp_anomaly_post_1850.loc[df_temp_anomaly_post_1850['Years'] >= -64]

#print(df_temp_anomaly_post_1959)

corr_co2_temp_anomaly_post_1959 = np.corrcoef(df_mauna_loa_csv['CO2'], df_temp_anomaly_post_1959['anomaly'])[0,1]

round_corr_co2_temp_anomaly_post_1959 = round(corr_co2_temp_anomaly_post_1959, 3)

print('The correlation coefficient between CO2 levels measured at Mauna Loa between 1959 and 2022')
print('and the Global temperature anomaly over the same time period is:')
print(f'{round_corr_co2_temp_anomaly_post_1959}')

Next I will create a sub-dataframe for CO2 which shifts the values for CO2 back one-year; I can then test the correlation of CO2 in 1960 with the value for temperature anomaly in 1961, and so on:

In [ ]:
# First I will merge the two dataframes

df_mauna_loa_temp_anomaly_post_1959 = pd.merge(df_mauna_loa_csv, df_temp_anomaly_post_1959, on = 'Years', how = 'outer')
#print(df_mauna_loa_temp_anomaly_post_1959)

# Here I am aligning the CO2 values for 1959 with the anomaly values for 1960, and onwards
# and adding a column to the merged dataframe above. The .shift() method shifts every value in the CO2 
# column forward one index
df_mauna_loa_temp_anomaly_post_1959['CO2_lead_1'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(1)

# To make it possible to calculate the correlation coefficient, I am removing the NaN data for 1959 and calculating
# the correlation between temperature anomaly and CO2 levels for the previous year for the data for 1960 to 2022 only
df_C02_lead_1 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] >= -63]

print(df_C02_lead_1)

corr_co2_temp_CO2_lead_1 = np.corrcoef(df_C02_lead_1['CO2_lead_1'], df_C02_lead_1['anomaly'])[0,1]

round_corr_co2_temp_CO2_lead_1 = round(corr_co2_temp_CO2_lead_1, 3)

print(round_corr_co2_temp_CO2_lead_1)

#print(df_mauna_loa_temp_anomaly_post_1959)

In [ ]:
# I will also test other lead and add similar columns 
df_mauna_loa_temp_anomaly_post_1959['CO2_lead_2'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(2)
df_CO2_lead_2 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] >= -62]
corr_co2_temp_CO2_lead_2 = np.corrcoef(df_CO2_lead_2['CO2_lead_2'], df_CO2_lead_2['anomaly'])[0,1]
round_corr_co2_temp_CO2_lead_2 = round(corr_co2_temp_CO2_lead_2, 3)

df_mauna_loa_temp_anomaly_post_1959['CO2_lead_3'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(3)
df_CO2_lead_3 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] >= -61]
corr_co2_temp_CO2_lead_3 = np.corrcoef(df_CO2_lead_3['CO2_lead_3'], df_CO2_lead_3['anomaly'])[0,1]
round_corr_co2_temp_CO2_lead_3 = round(corr_co2_temp_CO2_lead_3, 3)


# I will also test lag periods to test the correlation between temperature in an earlier year
# and carbon dioxide in a later year. I will remove years containing NaN frmo the end of the columns
df_mauna_loa_temp_anomaly_post_1959['CO2_lag_1'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(-1)
df_CO2_lag_1 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] <= -2]
corr_co2_temp_CO2_lag_1 = np.corrcoef(df_CO2_lag_1['CO2_lag_1'], df_CO2_lag_1['anomaly'])[0,1]
round_corr_co2_temp_CO2_lag_1 = round(corr_co2_temp_CO2_lag_1, 3)

df_mauna_loa_temp_anomaly_post_1959['CO2_lag_2'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(-2)
df_CO2_lag_2 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] <= -3]
corr_co2_temp_CO2_lag_2 = np.corrcoef(df_CO2_lag_2['CO2_lag_2'], df_CO2_lag_2['anomaly'])[0,1]
round_corr_co2_temp_CO2_lag_2 = round(corr_co2_temp_CO2_lag_2, 3)

df_mauna_loa_temp_anomaly_post_1959['CO2_lag_3'] = df_mauna_loa_temp_anomaly_post_1959['CO2'].shift(-3)
df_CO2_lag_3 = df_mauna_loa_temp_anomaly_post_1959.loc[df_mauna_loa_temp_anomaly_post_1959['Years'] <= -4]
corr_co2_temp_CO2_lag_3 = np.corrcoef(df_CO2_lag_3['CO2_lag_3'], df_CO2_lag_3['anomaly'])[0,1]
round_corr_co2_temp_CO2_lag_3 = round(corr_co2_temp_CO2_lag_3, 3)

# Before printing the correlation coefficients, I will print the dataframe containing all of
# the lead and lag columns
print(df_CO2_lag_3)

The correlations between leading or lagged CO2 levels and the temperature anomaly for the period 1959 to 2022 are calculated below:

In [ ]:
print('The correlation coefficients between the global temperature anomaly and ')
print('CO2 with a lead and lag of between 1 and 3 years are as follows:')
print()
print(f'Lead - 3 y:   {round_corr_co2_temp_CO2_lead_3}')
print(f'Lead - 2 y:   {round_corr_co2_temp_CO2_lead_2}')
print(f'Lead - 1 y:   {round_corr_co2_temp_CO2_lead_1}')
print(f'No lead/lag:  {round_corr_co2_temp_anomaly_post_1959}')
print(f'Lag  - 1 y:   {round_corr_co2_temp_CO2_lag_1}')
print(f'Lag  - 2 y:   {round_corr_co2_temp_CO2_lag_2}')
print(f'Lag  - 3 y:   {round_corr_co2_temp_CO2_lag_3}')

This shows that the highest correlation, perhaps suprisingly, exists between the temperature anomaly and the carbon dioxide level two years later. As calculated earlier, the correlation between CO2 levels and the temperature anomaly on a same-year basis between 1959 and 2022 was calculated as 0.378; this is in the middle of the range of correlation coefficients calculated by the lead-lag analysis.

My guess was that a higher CO2 level today can be expected to lead to a higher temperature over time, and that a lead of perhaps 1 year would show the highest correlation. It should be noted that for the lag analyses, between 1 and 3 columns of the highest CO2 data remain present to be correlated against a stationary temperature anomaly column, and this may cause the correlation to appear stronger than would be the case without the generally upward trend in carbon dioxide levels.

Even taking this into account, the correlation when there is no lead or lag between the two variables seems to be stronger than the correlation calculated for a one year lead or lag. Overall, the correlation of 0.378 is a moderate positive correlation, but it suggests that there are other factors which contributed to the global temperature anomaly of the past 64 years.

Rather than correlating against the absolute amount of CO2 in the atmosphere, it might be interesting to correlate the percentage change in CO2 levels year-on-year versus the percentage change in temperature anomaly year-on-year, or the correlation between a CO2 anomaly (measured against an appropriate CO2 baseline period) and the temperature anomaly. 

### CH4 versus Temperature Anomaly ###

#### Plotting CH4 versus Temperature Anomaly ####

Methane is a potent greenhouse gas and the addition of methane into the atmosphere from human economic activity is known to be contributing to atmospheric warming and climate change. According to the International Energy Agency, methane is responsible for around 30% of the  rise in global temperatures since the industrial revolution. 

It has a 'Global Warming Potential' (GWP) 84 times greater than CO2 (the reference gas used for calculating GWP) over a 20-year period, meaning a tonne of methane would absorb as much infrared thermal radiation as 84 tonnes of CO2; therefore, it's release into the atmosphere is 84 times more potent than an equivilant quantity released of CO2 over that timespan. However, methane has a shorter lifespan in the atmosphere than CO2, so its GWP over much longer timespans (eg. centuries) is lower than its GWP over decades. 

By way of comparison with the 20-year GWP value of 84 for methane, the 20-year GWP figures for Dichlorodifluoromethane (a CFC responsible for depletion of the ozone layer) is 10800, which highlights how it important it was that the manufacture of CFCs such as this was banned globally in 2010 (although recent studies have found that some CFCs are still being produced illegally in China).

I can use the composite dataset for atmospheric methane concentrations, which I created earlier, to plot the composite Antarctic temperature anomaly against inferred (for paleo) and recorded (for recent) methane concentrations in the atmosphere:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CH4 on the primary y-axis
ax1.plot(df_ch4['Years'], df_ch4['CH4'], color='purple', label='CH4')
ax1.set_xlabel('Years')
ax1.set_ylabel('CH4 (ppbv)', color='purple')
ax1.tick_params(axis='y', colors='purple')   

# Plot for temperature anomaly on secondary y-axis. 
ax2.plot(df_temp_anomaly['Years'], df_temp_anomaly['anomaly'], color='red', label='Temperature anomaly')
ax2.set_ylabel('Temperature Anomaly (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('Atmospheric CH4 levels (parts per billion by volume) versus Antarctic temperature anomaly from 800000 years ago to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CH4_vs_temp_anomaly_820000_ya.png')
plt.show()

As was the case when I plotted the composite CO2 data against the composite Antarctic temperature anomaly, it seems that local peaks in the data for CH4 are accompanied by local peaks in the data for the temperature anomaly. The most recent large spike in the temperature anomaly data corresponds to the Eemian interglacial, which occured between 115000 and 130000 years ago: during this period, average temperature was around 1 to 2 °C warmer than today.

I thought it would be interesting to plot the relationship between methane concentrations since 1000 AD with the temperature anomaly over that timespan, to get a closer picture of the relationship during more recent times:

In [ ]:
# First I will create a sub-dataframe from the CH4 dataframe containing only data relating to 1000 AD onwards
df_CH4_post_1000 = df_ch4.loc[df_ch4['Years'] >= - 1023]

# Next I will create a sub-dataframe of the anarctic temperature anomoly dataframe containg only data relating to
# 1000 AD onwards
df_temp_anomaly_post_1000 = df_temp_anomaly.loc[df_temp_anomaly['Years'] >= - 1023]

# Next, I will plot them against each other in the same way I did for the full 800000 years:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CH4 on the primary y-axis
ax1.plot(df_CH4_post_1000['Years'], df_CH4_post_1000['CH4'], color='purple', label='CH4')
ax1.set_xlabel('Years')
ax1.set_ylabel('CH4 (ppbv)', color='purple')
ax1.tick_params(axis='y', colors='purple')   

# Plot for temperature anomaly on secondary y-axis. 
ax2.plot(df_temp_anomaly_post_1000['Years'], df_temp_anomaly_post_1000['anomaly'], color='red', label='Temperature anomaly')
ax2.set_ylabel('Temperature Anomaly (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('Atmospheric CH4 levels ((parts per billion by volume) versus Antarctic temperature anomaly from 1000 AD to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CH4_vs_temp_anomaly_post_1000AD.png')
plt.show()


Some points to note from this plot:

- We can see that methane levels in the atmosphere are estimated to have been very steady at just under 700 parts per billion by volume in the atmosphere over the period 1000 AD to 1750 AD. After this time, methane levels began to increase, with the rate of increase accelerating over time. Levels are now over 1900 ppbv, meaning there is now 2.5 times more methane in the atmosphere than there was in the mid 1700s. 

- The temperature anomaly data can clearly be seperated into two sections; the first section corresponds to the sparser data which used the average temperature for 1000 AD to 1845 AD as a baseline. The second section plots the more granular annual temperature anomaly data which uses the period 1910 to 2000 as a baseline. As the period from roughly the 1500s to the early 1800s had lower temperatures than the rest of the millenium (the 'Little Ice Age'), it follows that the the average temperature over the period 1000 AD to 1845 AD was lower than the average temperature for the past century. As a result, temperature anomaly data for the first half of the millenium is more likely to be above 0.5°C than the temperature anomaly data for the last century, since it is being measured against a lower base figure.

We can also plot the methane data from 1850 onwards against the temperature anomaly data from 1850 onwards:

In [ ]:
# First I will create a sub-dataframe from the CH4 dataframe containing only data relating to 1850 AD onwards
df_CH4_post_1850 = df_ch4.loc[df_ch4['Years'] >= -173]

# Next, I will plot them against each other in the same way I did for the full 800000 years:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CH4 on the primary y-axis
ax1.plot(df_CH4_post_1850['Years'], df_CH4_post_1850['CH4'], color='purple', label='CH4')
ax1.set_xlabel('Years')
ax1.set_ylabel('CH4 (ppbv)', color='purple')
ax1.tick_params(axis='y', colors='purple')   

# Plot for temperature anomaly on secondary y-axis. 
ax2.plot(df_antarctic_temp_anomaly_1850['Years'], df_antarctic_temp_anomaly_1850['anomaly'], color='red', label='Temperature anomaly')
ax2.set_ylabel('Temperature Anomaly (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('Atmospheric CH4 levels (parts per billion by volume) versus Antarctic temperature anomaly from 1850 AD to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CH4_vs_temp_anomaly_post_1850.png')
plt.show()

Here we can see that the period of higher temperatures is associated with higher levels of methane in the atmosphere.


#### Correlation ####

To calculate the correlation coefficient between CH4 and the global temperature anomaly I must have two datasets of similar sizes, sharing a coming column. As was the case for CO2, this is a very difficult task using the older datasets; therefore, I will take the dataset of recent CH4 data, which goes back to 1984, and correlate this against a sub-dataframe of the temperature anomaly data which also goes back to 1984:

In [ ]:
df_temp_anomaly_post_1984 = df_temp_anomaly_post_1850.loc[df_temp_anomaly_post_1850['Years'] >= -39]

corr_ch4_temp_anomaly_post_1984 = np.corrcoef(df_ch4_recent['CH4'], df_temp_anomaly_post_1984['anomaly'])[0,1]

round_corr_ch4_temp_anomaly_post_1984= round(corr_ch4_temp_anomaly_post_1984, 3)

print('The correlation coefficient between CH4 levels measured between 1984 and 2022')
print('and the Global temperature anomaly over the same time period is:')
print(f'{round_corr_ch4_temp_anomaly_post_1984}')

This figure for the correlation coefficient between CH4 levels and the temperature anomaly since 1984 is quite weak in comparison with the figures calculated for CO2. 

From looking at the plot of CH4 versus temperature anomaly since 1850, it seems that, in percentage terms, the figure for methane in the atmosphere has not risen hugely compared to the figures for temperate anomaly. A manual check of the source data file shows that the figure for 1984 was 1644.85 ppbv and the figure for 2022 was 1911.83 ppbv, which equals an increase of 16.23%, and the rise was a steady one in between. Meanwhile, the temperature anomaly has risen (as calculated numerically, rather than in real-world terms) over 450% from (0.19 °C to 0.91 °C) over the same period, and has flucatuated a lot during that time.

Bearing in mind that the dataset only compared 39 years of data, this figure does not show a significant correlation between methane levels and the global temperature anomaly. As I talked about in the correlation and lead-lag regression section for CO2 vs Temperature anomaly, there may be a better way to test for a relationship than using absolute methane levels; instead, the percentage increase in methane vs some other function of the temperature anomaly might show a better correlation, if a true relationship between methane and the temperature anomaly exists.

I will not carry out a lead-lag analysis using this absolute methane data as the very low correlation I have found for the year-on-year data here suggests to me that the results will not be informative.

### CH4 versus CO2 ###

#### Plotting CH4 versus CO2 ####

Below I am going to plot long term CH4 levels against long term CO2 levels on the same x-axis to see if any relationship is immediately visible:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CH4 on the primary y-axis
ax1.plot(df_ch4['Years'], df_ch4['CH4'], color='purple', label='CH4')
ax1.set_xlabel('Years')
ax1.set_ylabel('Methane (ppbv)', color='purple')
ax1.tick_params(axis='y', colors='purple')   

# Plot for CO2 on secondary y-axis. 
ax2.plot(df_co2['Years'], df_co2['CO2'], color='blue', label='CO2')
ax2.set_ylabel('Carbon Dioxide (ppmv)', color='blue')
ax2.tick_params(axis='y', colors='blue')

plt.title('Atmospheric CH4 levels (ppbv) versus CO2 levels (ppmv) from 800000 years ago to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/CH4_vs_CO2.png')
plt.show()

Interestingly, there seems to be relationship between these two variables, with local peaks in CO2 associated with local peaks in CH4. 

### Correlation and lead-lag relationship ###

I would like to calculate the correlation coefficient between the full datasets for CH4 and CO2; unfortunately, as was the case for the correlations between temperature anomaly and each of these variables, it is very difficult to align such large datasets without introducing randomness to the final result by taking samples and needing to find a way to get around the fact that year numbers don't align due to the presence of decimal precision.

Instead, I will take the furthest-back common dataset I can get which will align both variables to test the correlation between them. I will take the Mauna Loa CO2 data back to 1984 and correlate this against the recent CH4 data:

In [ ]:
df_CO2_post_1984 = df_mauna_loa_csv.loc[df_mauna_loa_csv['Years'] >= -39]

corr_CO2_CH4_post_1984 = np.corrcoef(df_CO2_post_1984['CO2'], df_ch4_recent['CH4'])[0,1]

round_corr_CO2_CH4_post_1984 = round(corr_CO2_CH4_post_1984, 3)

print('The correlation coefficient between CO2 levels measured at Mauna Loa between 1984 and 2022')
print('and CH4 levels measured over the same time period is:')
print(f'{round_corr_CO2_CH4_post_1984}')

This produces a very strong correlation of 0.969, indicating that these two variables are highly associated with one another. I will now carry out a lead-lag analysis like I did earlier:

In [ ]:
# First I will merge the two dataframes
df_CO2_CH4_post_1984 = pd.merge(df_CO2_post_1984, df_ch4_recent, on = 'Years', how = 'inner')

# Here I am aligning the CO2 values for 1984 with the CH4 values for 1985, and onwards
# and adding a column to the merged dataframe above. The .shift() method shifts every value in the CO2 
# column forward one index
df_CO2_CH4_post_1984['CO2_lead_1'] = df_CO2_CH4_post_1984['CO2'].shift(1)

# To make it possible to calculate the correlation coefficient, I am removing the NaN data for 1984 and calculating
# the correlation between CH4 and CO2 levels for the previous year for the data for 1985 to 2022 only
df_C02_CH4_lead_1 = df_CO2_CH4_post_1984[df_CO2_CH4_post_1984['Years'] >= -38]

print(df_C02_CH4_lead_1)

corr_CO2_CH4_lead_1 = np.corrcoef(df_C02_CH4_lead_1['CO2_lead_1'], df_C02_CH4_lead_1['CH4'])[0,1]

round_corr_CO2_CH4_lead_1 = round(corr_CO2_CH4_lead_1, 3)

print(round_corr_CO2_CH4_lead_1)

#print(df_mauna_loa_temp_anomaly_post_1959)

The correlation of 0.973 between the carbon dioxide level of the previous year and the methane level of the current year is slightly higher than the correlation between the levels on the same year (0.969). The correlations for a lead and lags of up to 3-years are calculated below:

In [ ]:
# I will also test other lead and add similar columns 
df_CO2_CH4_post_1984['CO2_lead_2'] = df_CO2_CH4_post_1984['CO2'].shift(2)
df_CO2_CH4_lead_2 = df_CO2_CH4_post_1984.loc[df_CO2_CH4_post_1984['Years'] >= -37]
corr_CO2_CH4_lead_2 = np.corrcoef(df_CO2_CH4_lead_2['CO2_lead_2'], df_CO2_CH4_lead_2['CH4'])[0,1]
round_corr_CO2_CH4_lead_2 = round(corr_CO2_CH4_lead_2, 3)


df_CO2_CH4_post_1984['CO2_lead_3'] = df_CO2_CH4_post_1984['CO2'].shift(3)
df_CO2_CH4_lead_3 = df_CO2_CH4_post_1984.loc[df_CO2_CH4_post_1984['Years'] >= -36]
corr_CO2_CH4_lead_3 = np.corrcoef(df_CO2_CH4_lead_3['CO2_lead_3'], df_CO2_CH4_lead_3['CH4'])[0,1]
round_corr_CO2_CH4_lead_3 = round(corr_CO2_CH4_lead_3, 3)


# I will also test lag periods to test the correlation between temperature in an earlier year
# and carbon dioxide in a later year. I will remove years containing NaN frmo the end of the columns
df_CO2_CH4_post_1984['CO2_lag_1'] = df_CO2_CH4_post_1984['CO2'].shift(-1)
df_CO2_CH4_lag_1 = df_CO2_CH4_post_1984.loc[df_CO2_CH4_post_1984['Years'] <= -2]
corr_CO2_CH4_lag_1 = np.corrcoef(df_CO2_CH4_lag_1['CO2_lag_1'], df_CO2_CH4_lag_1['CH4'])[0,1]
round_corr_CO2_CH4_lag_1 = round(corr_CO2_CH4_lag_1, 3)

df_CO2_CH4_post_1984['CO2_lag_2'] = df_CO2_CH4_post_1984['CO2'].shift(-2)
df_CO2_CH4_lag_2 = df_CO2_CH4_post_1984.loc[df_CO2_CH4_post_1984['Years'] <= -3]
corr_CO2_CH4_lag_2 = np.corrcoef(df_CO2_CH4_lag_2['CO2_lag_2'], df_CO2_CH4_lag_2['CH4'])[0,1]
round_corr_CO2_CH4_lag_2 = round(corr_CO2_CH4_lag_2, 3)

df_CO2_CH4_post_1984['CO2_lag_3'] = df_CO2_CH4_post_1984['CO2'].shift(-3)
df_CO2_CH4_lag_3 = df_CO2_CH4_post_1984.loc[df_CO2_CH4_post_1984['Years'] <= -4]
corr_CO2_CH4_lag_3 = np.corrcoef(df_CO2_CH4_lag_3['CO2_lag_3'], df_CO2_CH4_lag_3['CH4'])[0,1]
round_corr_CO2_CH4_lag_3 = round(corr_CO2_CH4_lag_3, 3)

# Before printing the correlation coefficients, I will print the dataframe containing all of
# the lead and lag columns
print(round_corr_CO2_CH4_lead_3)

The correlations between leading or lagged CO2 levels and CH4 for the period 1984 to 2022 are calculated below:

In [ ]:
print('The correlation coefficients between CH4 and CO2 with ')
print('a lead and lag of between 1 and 3 years are as follows:')
print()
print(f'Lead - 3 y:   {round_corr_CO2_CH4_lead_3}')
print(f'Lead - 2 y:   {round_corr_CO2_CH4_lead_2}')
print(f'Lead - 1 y:   {round_corr_CO2_CH4_lead_1}')
print(f'No lead/lag:  {round_corr_CO2_CH4_post_1984}')
print(f'Lag  - 1 y:   {round_corr_CO2_CH4_lag_1}')
print(f'Lag  - 2 y:   {round_corr_CO2_CH4_lag_2}')
print(f'Lag  - 3 y:   {round_corr_CO2_CH4_lag_3}')

The results above are interesting, as they imply that the correlation between CO2 levels three-years ago and CH4 levels today is slightly stronger than the correlation between CO2 and CH4 levels today. The difference between each correlation coefficient is slight, and all figures indicate an almost perfect correlation, but the pattern is that the correlation weakens slightly when future CO2 levels are regressed against current CH4 and it strenghtens slightly when past C02 levels are regressed against current CH4. 

My interpretation of these very strong correlations is simply that the same cause is driving the increase in both - constantly growing economic activity underpinned by burning fossil fuels for energy.

### Climate change signals in the Irish context ###

#### Background to Irish climate analysis ####

I will now investigate the relationship between global increases in temperature and more local climate conditions. In December 2023, a paper on this topic written by a group of Irish climate scientists appeared in the journal 'Weather and Climate Extremes'; the title of the paper is "The emergence of a climate change signal in long-term Irish meteorological observation". The authors used composite data for temperature and rainfall for the island of Ireland to compare seasonal and annual means and extremes for this data, plus pre-defined climate indices based on the data, against the corresponding data for the base period 1961 to 1990 (which they call Standard Climate Normal, or SCN) on the island of Ireland. They used 14 temperature indices and 5 rainfall indices.

To determine how different modern climate conditions are to the conditions during this base period, they used a "signal to noise ratio"("SNR"), where seperates changes in climate data which are due to a long-term trend versus normal variation in weather which could be expected independent of the presence of any long-term trend.

They calculated how unusual conditions would be compared to today, with an SNR < 1 meaning 'normal or familiar', SNR > 1 and < 2 meaning 'unusual', SNR > 2 and < 3 meaning 'unfamiliar' and SNR > 3 meaning 'unknown'. 

For my analysis, I am going to take a more straightforward approach. I have obtained reliable monthly data going back to 1950 for one of Met Eireann's primary weather observing stations, Mullingar in Co. Westmeath, and climate average data for the period 1961 to 1990 for this same weather station. The period 1961 to 1990 is commonly used as a "Standard Normal Reference" period in weather analysis to assess long-term climate change, including by Met Éireann.

Mullingar is a town in midlands of Ireland. The climate there might be considered to represent the midlands as opposed to the coastal regions, but this weather station is as good as most to give a picture of a the typical weather at a weather station in Ireland which isn't very close to the coast.

<img src="./images/mullingar_on_map.jpg" alt="Mullingar on map of Ireland">

I chose the Mullingar station because it has a full dataset for the time in question, including eleven years of data from before the period of 1961 to 1990. I also know from earlier work in this project that the global temperature anomaly didn't ramp up until around the 1950s - 1960s, so I feel the trade off for not including pre-1940 Irish weather data (which can be difficult to find and can be unreliable) is worth it for the sake of simplifying the task of seeking the presence of climate change signals in the Irish climate. For the same reason, I have chosen one weather station rather than multiple; I am hoping that by calculating the anomaly against the average of a 30 year reference period, and plotting 73 data points which themselves represent annual averages, some of the background variability in the actual weather data over the 73-year period can be smoothed out and the presence or not of a trend linked to wider climate change could be discerned.

I have manually selected the data I want to use from the source file for the 1961 to 1990 average data and saved it as a csv file in my working directory, along with a trimmed version of the csv file containing the monthly weather data for Mullingar from 1950 to 2022. 

I will work out the mean annual temperature and mean annual total rainfall for each year from 1950 to 2022 and then use the mean annual data for temperature and rainfall for the period 1961 - 1990 to calculate the annual temperature and rain anomaly for each year from 1950 to 2022 file (the source file contained monthly data running until November 2023, but to avoid running into issues if I needed to calculate averages based on twelve months, I decided to leave out 2023). I will plot this anomaly over time, and alongside the global temperature anomaly for the same period.

Temperatures are in degrees Celsius for the month and rainfall is measured in total millimetres per month. Each month in the dataset is coded from 1 to 12, corresponding to January to December. I have included sources for my data in my references.

#### Calculating temperature and rainfall anomalies ####

Firstly, I will upload and print the figures for mean monthly temperature and total monthly rainfall for each month of the year, calculated for the period 1961 to 1990. 

Then, I will calculate the mean annual temperature for this period based on adding up the twelve figures for mean temp for each month and dividing this sum by twelve ie. finding the mean figure. This figure is like a "mean of means" and I will use this to calculate my temperature anomaly against the 1950 to 2022 data.

For rainfall, I will **sum** the mean total rainfall for each of the twelve months to produce a figure for the mean **total annual** rainfall amount for a year between 1961 and 1990. I will use this figure to calculate my rainfall anomaly against the 1950 to 2022 data.

*Mean temperature and rainfall 1961 to 1990:*

In [ ]:
df_mullingar_means_1961_1990 = pd.read_csv("climate_data/mullingar_mean_temp_rain_1961_1990_csv.csv")

print(df_mullingar_means_1961_1990)

mean_annual_temp_1961_to_1990 = df_mullingar_means_1961_1990['mean temp'].mean()
round_mean_annual_temp_1961_to_1990 = round(mean_annual_temp_1961_to_1990, 1)

mean_annual_rain_1961_to_1990 = df_mullingar_means_1961_1990['mean total rainfall'].sum()
round_mean_annual_rain_1961_to_1990 = round(mean_annual_rain_1961_to_1990, 1)

print()
print(f'The mean annual temperature at Mullingar for 1961 to 1990 is {round_mean_annual_temp_1961_to_1990} °C')
print(f'The mean annual total rainfall at Mullingar for 1961 to 1990 is {round_mean_annual_rain_1961_to_1990} millimetres of rain')


Next, I will create two new sub-dataframes:

- One containing the mean annual temperature for each year in the source file.
- One containing the mean annual total rainfall for each year in the source file.

I will use the figures obtained above for the 1961 to 1990 period of 8.8 °C for the mean annual temperature and 934.3 mm for the main annual total rainfall to calculate the temperature and rainfall anomalies for each year from 1950 to 2022.

I will then merge the Mullingar temperature and rainfall data for each year into one dataframe and add a column called 'Years' to make it possible to merge this data with other datasets later on.

In [ ]:
df_mullingar_monthly_temp_rain = pd.read_csv("climate_data/mullingar_mean_temp_rain_1950_2022_csv.csv")

# Here I create a new dataframe with mean annual temperature for each year. The reset_index method ensures the 
# figures for means are added to the dataframe with the same index as the year number associated with it
df_mean_annual_temp_1950_to_2022 = df_mullingar_monthly_temp_rain.groupby('year')['mean_temp'].mean().reset_index()

# Create a new dataframe with annual total rainfall for each year
df_annual_rain_1950_to_2022 = df_mullingar_monthly_temp_rain.groupby('year')['mean_rain'].sum().reset_index()

# Here I have used .loc[:, 'temp_anomaly'] to create a new column which has a value for all the rows which are present
# in the mean annual temperature sub-dataframe. I am using this syntax to avoid getting an warning 
# "A value is trying to be set on a copy of a slice from a DataFrame"
df_mean_annual_temp_1950_to_2022.loc[:, 'temp_anomaly'] = df_mean_annual_temp_1950_to_2022['mean_temp'] - 8.8

# Here I am doing similar to above, but creating a column for the rainfall anomaly instead
df_annual_rain_1950_to_2022.loc[:, 'rain_anomaly'] = df_annual_rain_1950_to_2022['mean_rain'] - 934.3

# now I want to merge these two newly created dataframes into one dataframe, joined by the shared column 'year'
df_mullingar_weather = pd.merge(df_mean_annual_temp_1950_to_2022, df_annual_rain_1950_to_2022, on = 'year', how ='inner')

# I will also add a column called 'Years' to facillitate merging this dataframe with my other dataframes later on
# on the basis of a shared column
df_mullingar_weather['Years'] = (df_mullingar_weather['year'] - 2023) * 1

print(df_mullingar_weather)

#### Plotting Mullingar climate anomalies ####

Now I can plot the temperature and rainfall anomalies over time. I have also included linear regression equations for each anomaly to predict future years based on the assumption of the current trend continuing on in a linear fashion:

*Temperature anomaly:*

In [ ]:
from scipy.stats import linregress 

plt.figure(figsize= (16,6))
plt.xlabel('Years')
plt.ylabel('Temperature Anomaly (°C))')
plt.title('Temperature anomaly (vs 1961 to 1990 average) in °C at Mullingar weather station - 1950 to 2022')

sns.lineplot(x ='year', y = 'temp_anomaly', data = df_mean_annual_temp_1950_to_2022, color = 'blue')

sns.regplot(x='year', y='temp_anomaly', data=df_mean_annual_temp_1950_to_2022, line_kws={'color': 'red'}, scatter_kws={'s': 5})

plt.savefig("images/Mullingar_temp_anomaly.png")

slope, intercept, r_value, p_value, std_err = linregress(df_mean_annual_temp_1950_to_2022['year'], df_mean_annual_temp_1950_to_2022['temp_anomaly'])

print(f'The linear regression equation for this line is (temp_anomaly) = ({slope})*(year) + ({intercept})')


*Rainfall anomaly:*

In [ ]:
plt.figure(figsize= (16,6))
plt.xlabel('Years')
plt.ylabel('Rainfall anomaly (mm per year)')
plt.title('Rainfall anomaly (vs 1961 to 1990 average) in millimetres per year at Mullingar weather station - 1950 to 2022')

sns.lineplot(x ='year', y ='rain_anomaly', data = df_annual_rain_1950_to_2022, color = 'green')

sns.regplot(x ='year', y ='rain_anomaly', data=df_annual_rain_1950_to_2022, line_kws={'color': 'red'}, scatter_kws={'s': 5})

plt.savefig("images/Mullingar_rainfall_anomaly.png")

slope, intercept, r_value, p_value, std_err = linregress(df_annual_rain_1950_to_2022['year'], df_annual_rain_1950_to_2022['rain_anomaly'])

print(f'The linear regression equation for this line is (rain_anomaly) = ({slope})*(year) + ({intercept})')


We can see from the plots above that there is a slight upward trend the in the temperature anomaly and rainfall anomaly for Mullingar weather station between 1950 and 2022. The trend is slightly stronger for the temperature anomaly.

#### Mullingar temperature anomaly versus Global temperature anomaly ####

I will now plot the Mullingar temperature anomaly and the global temperature anomaly on the same x and y-axes, to get a sense of how their trends compare over the period 1950 to 2022. It should be remembered, however, that the base period used to calculate the Mullingar temperature anomalies is the period 1961 to 1990, while for the global dataset the period 1901 to 2000 is used; since humanity's impact on heating the planet was not as great in the first fifty years of the 20th century compared to the second fifty, I am hoping that the use of the different base periods between the two samples does not reduce the validity of the information displayed by the plots:

In [ ]:
# First I will create a sub-dataframe from the global temperature anomaly dataframe containing only data relating to 1950 onwards
df_global_temp_anomaly_post_1950 = df_global_temp_anomaly_1850.loc[df_global_temp_anomaly_1850['Years'] >= - 73]

# Next I will plot the temperature anomaly from this global dataframe and the Mullingar dataframe against each other
# plt.figure(figsize= (16,6))

fig, ax1 = plt.subplots(figsize=(16, 6))

sns.lineplot(x='Years', y='temp_anomaly', data=df_mullingar_weather, color = 'blue', label='Mullingar temp anomaly')
sns.lineplot(x='Years', y='anomaly', data=df_global_temp_anomaly_post_1950, color = 'red', label='Global temp anomaly')

plt.title('Mullingar temperature anomaly vs Global temperature anomaly - 1950 to 2022')
plt.xlabel('Years ago')
plt.ylabel('Temperature Anomaly (°C)')

plt.legend()

plt.savefig('images/Mullingar_vs_Global_temp_anomaly.png')

plt.show()

corr_mullingar_global_temp_anomaly = np.corrcoef(df_mullingar_weather['temp_anomaly'], df_global_temp_anomaly_post_1950['anomaly'])[0,1]
round_corr_mullingar_global_temp_anomaly = round(corr_mullingar_global_temp_anomaly, 2)
print(f'The correlation coefficient between the temperature and rainfall anomalies in Mullingar is {round_corr_mullingar_global_temp_anomaly}')

While the data for Mullingar shows greater swings than the Global data, a positive correlation appears to be present between both sets of data, where the increase in the global temperature anomaly over time occurs during the same time as an increase in the Mullingar temperature anomaly. Indeed, the correlation coefficient between the two sets of data shows a value of 0.5, representing a moderate positive correlation.

#### Mullingar temperature anomaly versus rainfall anomaly ####

I was curious to test if any correlation existed between the rainfall and temperature anomalies, so I will now plot the temperature and rainfall anomalies together on the same x-axis:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for temperature anomaly on the primary y-axis
ax1.plot(df_mullingar_weather['year'], df_mullingar_weather['temp_anomaly'], color='red', label='Temperature anomaly')
ax1.set_xlabel('Years')
ax1.set_ylabel('Temperature anomaly (°C)', color='red')
ax1.tick_params(axis='y', colors='red')   

# Plot for temperature on secondary y-axis
ax2.plot(df_mullingar_weather['year'], df_mullingar_weather['rain_anomaly'], color='green', label='Rainfall anomaly')
ax2.set_ylabel('Rainfall anomaly (mm)', color='green')
ax2.tick_params(axis='y', colors='green')

plt.title('Temperature anomaly vs Rainfall anomaly for Mullingar weather station - 1950 to 2023') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('images/Mullingar_temp_anomaly_vs_rain_anomaly.png')
plt.show()

corr_temp_rain_anomaly = np.corrcoef(df_mullingar_weather['temp_anomaly'], df_mullingar_weather['rain_anomaly'])[0,1]
round_corr_temp_rain_anomaly = round(corr_temp_rain_anomaly, 2)
print(f'The correlation coefficient between the temperature and rainfall anomalies in Mullingar is {round_corr_temp_rain_anomaly}')

From looking at the plot, it is difficult to discern any clear relationship between these variables in Mullingar over the timespan examined, and the correlation coefficient of -0.02 confirms an almost complete lack of relationship between the temperature anomaly and rainfall anomaly over time. This is an interesting finding in itself, because I did not have any intuition either way as to how they might be correlated, so it seems they may not be strongly correlated at all (in a local context in any case).

### Fusion of datasets ###

I will now fuse my datasets for CO2, CH4, temperature and rainfall into one composite dataframe, before exporting it to csv and json files. For the Mullingar, Antarctic temperature anomaly and Global temperature anomaly dataframes, I will rename some columns to make it clearer in the composite dataframe what each column of data represents. I will also change the variable type of some columns from integers to floats to avoid getting warnings for fusing types in columns. Below are the datasets I will fuse together:

In [ ]:
print(df_co2)

In [ ]:
print(df_ch4)

In [ ]:
#print(df_mullingar_weather)

df_mullingar_weather.rename(columns={'mean_temp': 'mullingar_mean_temp', 'temp_anomaly': 'mullingar_temp_anomaly', 'mean_rain': 'mullingar_mean_rain', 'rain_anomaly': 'mullingar_rain_anomaly', 'Years': 'Years'}, inplace=True) 

# converting integers to floats in 'Years' column to avoid getting warnings for mergings ints and floats
df_mullingar_weather['Years'] = df_mullingar_weather['Years'].astype(float)

# Here I am dropping the unwanted column 'year'. The axis = 1 parameter is used to specify that it is a column being
# dropped as opposed to a row
df_mullingar_weather = df_mullingar_weather.drop(['year'], axis = 1)

print(df_mullingar_weather)

In [ ]:
df_temp_anomaly.rename(columns={'Years': 'Years', 'anomaly': 'antarctic_temp_anomaly'}, inplace=True) 

print(df_temp_anomaly)

In [ ]:
df_global_temp_anomaly_1850.rename(columns={'anomaly': 'global_temp_anomaly', 'Years': 'Years'}, inplace=True) 

# converting integers to floats in 'Years' column to avoid getting warnings for mergings ints and floats
df_global_temp_anomaly_1850['Years'] = df_global_temp_anomaly_1850['Years'].astype(float)

# Here I am dropping the unwanted column 'year'. The axis = 1 parameter is used to specify that it is a column being
# dropped as opposed to a row
df_global_temp_anomaly_1850 = df_global_temp_anomaly_1850.drop(['year'], axis = 1)

print(df_global_temp_anomaly_1850)

In [ ]:
# Here I am merging each dataframe one at a time to the others based on them sharing the common column 'Years'
df_co2_ch4 = pd.merge(df_co2, df_ch4, on = 'Years', how = 'outer')

df_co2_ch4_mullingar = pd.merge(df_co2_ch4, df_mullingar_weather, on = 'Years', how = 'outer')

df_co2_ch4_mullingar_temp_anomaly = pd.merge(df_co2_ch4_mullingar, df_temp_anomaly, on = 'Years', how = 'outer')

df_merged = pd.merge(df_co2_ch4_mullingar_temp_anomaly, df_global_temp_anomaly_1850, on = 'Years', how = 'outer')

# Here I am creating a new dataframe which is the df_merged dataframe sorted according to the value in the 'Years' column
df_merged_sorted = df_merged.sort_values(by = 'Years')

# reversing the order of the dataset, so that more recent years appear first when the data is exported
df_fused = df_merged_sorted[::-1]

print(df_fused)


I am happy with the dataframe I have created above and I want to export it now to CSV and JSON files:

Firstly, CSV:

In [ ]:
df_fused.to_csv('climate_data/composite climate data.csv', index=False)

Now, JSON:

In [ ]:
# use orient = 'records' parameter to turn each row into a list-like object in json format.
# 'Lines = True' paramater writes each outputted object onto its own line in the exported json file
df_fused.to_json('climate_data/composite climate data.json', orient='records', lines=True) 

### Prediction of Global Temperature Anomaly ###

I will now make predictions for the future of the global temperature anomaly, based on the assumption the trend in the data will continue in a linear fashion. I will examine the data from 1950, since the most significant increases in the anomaly have occured since then.

I will then compare these predictions with the predictions being made by climate scientists to investigate whether the trend for the temperature anomaly is accelerating rarther than increasingly linearly:

In [ ]:
# Here i am reshaping the list of x values as required by Sklearns LinearRegression class
x = df_global_temp_anomaly_post_1950['year'].values.reshape(-1, 1)
y = df_global_temp_anomaly_post_1950['anomaly']

lr = LinearRegression()
lr.fit(x, y)

# these are the x-values which will go into the linear-regression model
future_years = np.array([2025, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])

# I am reshaping the above array to enable the linear regression code to process the data
future_years_reshaped = future_years.reshape(-1, 1)

# these are the y-values (temp anomalies) which will be predicted by the linear regression model
future_anomalies = lr.predict(future_years_reshaped)

# I will print the list of predictions for the global temperature anomaly for each of the years contained in the
# 'future_years' array above
df_future_anomalies_by_year = pd.DataFrame({'year':future_years, 'anomaly':future_anomalies})
print()
print(df_future_anomalies_by_year)
print()

plt.figure(figsize= (14,6))
plt.title('Global temperature anomaly (in °C) since 1950 - observed data and prediction to 2100 based on linear regression')
plt.xlabel('Year')
plt.ylabel('Temperature Anomaly (°C)')
plt.scatter(df_global_temp_anomaly_post_1950['year'], df_global_temp_anomaly_post_1950['anomaly'])
plt.plot(future_years_reshaped, future_anomalies, color ='brown')
plt.legend(["Observed data", "Predicted line"])
plt.grid(True)
plt.savefig('images/Global_temp_anomaly_linear_prediction.png')
plt.show()

#print(df_global_temp_anomaly_post_1950)

This linear projection of Global temperature anomaly up to the year 2100 predicts the anomaly to reach 2 °C by that time. 

In 2021, the 'Summary for Policymakers' document was written up by many climate scientists for the Intergovernmental Panel on Climate Change (https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_SPM.pdf). In this document (on page 13), five different potential scenarios for CO2 emissions between 2021 and the end of the century were laid out and the consequences for global temperature for the period 2081 - 2100 were predicted for each scenario. Under the least damaging/ most optimistic scenario (called SSP1-1.9 in the document), temperatures are still expected to lie between 1 and 1.9 °C higher on average between 2081 - 2100 than they were betwen 1850 and 1900. For this illustrative scenario to come to pass, the world would need to reach a net-zero situation with regard to CO2 emissions by the 2050s and sustain it indefinitely. The least optimistic scenario sees humanity putting out a net 130 gigatons per year of CO2 by the end of the century (compared with the net figure of 37.55 gigatons produced in 2023) and temperatures in the period 2081 to 2100 being between 3.3 and 5.7 °C hotter than the period 1850 to 1900:

<img src="./images/IPCC_CO2_predictions.png" alt="IPCC CO2 scenarios">

The scenario I predicted for the year 2100 (using linear regression) produced an estimate of 2 °C for that year; such a scenario seems to lie within the second most optimistic scenario of the five scenarios for future temperature, as outlined in the 'Summary for Policymakers' document. This scenario sees a range for average global temperature for the period 2081 to 2100 of 1.3 to 2.4 °C higher than the base period 1850 to 1900. To secure such a scenario, net-zero CO2 emissions would need to be achieved globally by the 2070s, and concentrations of methane, nitrous oxide and sulphur dioxide will need to be reduced enormously too.

<img src="./images/temp_predictions.png" alt="IPCC temperature predictions">

### Accelerating increases of temperature ###

To test whether the rate of increase of the global temperature anomaly is itself increasing, I have downloaded data from the NOAA website for the global temperature anomaly for each month from January 2018 to November 2023 (most recent month available for that year). Each datapoint measures the temperature difference in °C during that month relative to the average for that month for the period 1901 - 2000.

*Temperature anomaly by year:*

To test if the rate of temperature increase is accelerating, I am going to divide this monthly dataset into the six years from 2018 to 2023 inclusive (although 2023 will only contain 11 months worth of data). I will then calculate the slope of the regression line associated with each year to see if the slopes are getting steeper; if it is, it would suggest that the rate of increase is itself increasing ie. the warming is accelerating.

In [ ]:
# Firstly, I will create sub-dataframes for each year of the recent global temperature anomaly dataset

df_global_temp_recent = pd.read_csv("climate_data/global_temp_recent_csv.csv")

df_2018_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2018]
df_2019_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2019]
df_2020_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2020]
df_2021_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2021]
df_2022_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2022]
df_2023_global_temp = df_global_temp_recent.loc[df_global_temp_recent['year'] == 2023]

Next, I will create a list of slope values by year by using the **linregress function** from scipy.stats (this also allows me to print other data associated with the linear regression equations, but for now I only wish to record the slope).

I will also plot a regression line for each year (however, due to the number of plots, I will not save them down to my subfolder 'images').

Attention should be paid to the temperature anomaly values on the y-axis and the steepness of the slope should be considered in conjunction with these figures.

In [ ]:
slope_2018, intercept, r_value, p_value, std_err = linregress(df_2018_global_temp['month'], df_2018_global_temp['anomaly'])
slope_2019, intercept, r_value, p_value, std_err = linregress(df_2019_global_temp['month'], df_2019_global_temp['anomaly'])
slope_2020, intercept, r_value, p_value, std_err = linregress(df_2020_global_temp['month'], df_2020_global_temp['anomaly'])
slope_2021, intercept, r_value, p_value, std_err = linregress(df_2021_global_temp['month'], df_2021_global_temp['anomaly'])
slope_2022, intercept, r_value, p_value, std_err = linregress(df_2022_global_temp['month'], df_2022_global_temp['anomaly'])
slope_2023, intercept, r_value, p_value, std_err = linregress(df_2023_global_temp['month'], df_2023_global_temp['anomaly'])

# rounding each value to 4 decimal places to enhance readibility
round_slope_2018 = round(slope_2018, 4)
round_slope_2019 = round(slope_2019, 4)
round_slope_2020 = round(slope_2020, 4)
round_slope_2021 = round(slope_2021, 4)
round_slope_2022 = round(slope_2022, 4)
round_slope_2023 = round(slope_2023, 4)

# creating two arrays of figures which I plot in a dataframe
slope_years = np.array([2018, 2019, 2020, 2021, 2022, 2023])
slope_calculations = np.array([round_slope_2018, round_slope_2019, round_slope_2020, round_slope_2021, round_slope_2022, round_slope_2023])

# creating a dataframe containing the years and accompanying values for slope of the regression line for the data
# for global temperature anomaly
df_regression_slopes_for_anomalies_by_year = pd.DataFrame({'year':slope_years, 'slope':slope_calculations})

print()
print(f'The slope of the regression line for the global temperature anomaly for each of the past six years is as follows:')
print()
print(df_regression_slopes_for_anomalies_by_year)
print()

# We can also create regplots for each year:
plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Dec 2018')
plt.xlabel('Months in 2018')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2018_global_temp, order=1, color = 'brown')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Dec 2019')
plt.xlabel('Months in 2019')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2019_global_temp, order=1, color = 'brown')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Dec 2020')
plt.xlabel('Months in 2020')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2020_global_temp, order=1, color = 'brown')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Dec 2021')
plt.xlabel('Months in 2021')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2021_global_temp, order=1, color = 'brown')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Dec 2022')
plt.xlabel('Months in 2022')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2022_global_temp, order=1, color = 'brown')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) Jan to Nov 2023')
plt.xlabel('Months in 2023')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='month', y='anomaly', data=df_2023_global_temp, order=1, color = 'brown')
plt.show()

We can see from the figures generated above that the slope of the regression line for the first eleven months of 2023 (0.0499) is steeper than the regression lines for each of the other five years (the next closest is 2021, when Covid lockdowns began to ease, with a figure of 0.0162). The figures for 2018 and 2019 were almost flat by comparison, and those of 2020 and 2022 were actually negative. This suggests that over the course of 2023, the figure for temperature anomaly was trending upwards.

*Mean temperature anomaly by year:*

Since a steep slope may carry less significance if the temperatures for each month were lower than average, I will calculate the mean anomaly for each year based on the monthly data for each year; this will hopefully give me a sense of how signifcant a high slope line is for 2023 compared to other years.

In [ ]:
# mean anomaly for each dataframe

global_temp_2018_mean = df_2018_global_temp['anomaly'].mean()
global_temp_2019_mean = df_2019_global_temp['anomaly'].mean()
global_temp_2020_mean = df_2020_global_temp['anomaly'].mean()
global_temp_2021_mean = df_2021_global_temp['anomaly'].mean()
global_temp_2022_mean = df_2022_global_temp['anomaly'].mean()
global_temp_2023_mean = df_2023_global_temp['anomaly'].mean()

global_temp_mean_years = np.array([2018, 2019, 2020, 2021, 2022, 2023])
mean_anomaly_calculations = np.array([global_temp_2018_mean, global_temp_2019_mean, global_temp_2020_mean, global_temp_2021_mean, global_temp_2022_mean, global_temp_2023_mean])

df_mean_anomalies_by_year = pd.DataFrame({'year':global_temp_mean_years, 'mean anomaly':mean_anomaly_calculations})
print('The mean global temperature anomaly (in °C) for each year is as below:')
print()
print(df_mean_anomalies_by_year)

We can see that, in addition to having the steepest regression line for its data for the global temperature anomaly, 2023 also had the highest average temperature anomaly out of the past six years. This adds to the evidence that 2023 may be a year of accelerating temperature growth.

*Temperature anomaly by month:*

I want to create sub-dataframes for each **month** of the year, to test whether the slopes of the regression lines for each month over the past six years are positive or negative ie. are individual months showing a gradual rise or fall in temperature, and, if so, to what extent.

Firstly, I will create sub-dataframes for each month of the recent global temperature anomaly dataset:

In [ ]:
# sub-dataframes for each month of the year
df_jan_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 1]
df_feb_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 2]
df_mar_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 3]
df_apr_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 4]
df_may_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 5]
df_jun_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 6]

df_jul_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 7]
df_aug_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 8]
df_sep_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 9]
df_oct_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 10]
df_nov_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 11]
df_dec_global_temp = df_global_temp_recent.loc[df_global_temp_recent['month'] == 12]

Next, I will print out a list of the slopes associated each month over the past six years. I will use the linregress function again:

In [ ]:
slope_jan, intercept, r_value, p_value, std_err = linregress(df_jan_global_temp['year'], df_jan_global_temp['anomaly'])
slope_feb, intercept, r_value, p_value, std_err = linregress(df_feb_global_temp['year'], df_feb_global_temp['anomaly'])
slope_mar, intercept, r_value, p_value, std_err = linregress(df_mar_global_temp['year'], df_mar_global_temp['anomaly'])
slope_apr, intercept, r_value, p_value, std_err = linregress(df_apr_global_temp['year'], df_apr_global_temp['anomaly'])
slope_may, intercept, r_value, p_value, std_err = linregress(df_may_global_temp['year'], df_may_global_temp['anomaly'])
slope_jun, intercept, r_value, p_value, std_err = linregress(df_jun_global_temp['year'], df_jun_global_temp['anomaly'])

slope_jul, intercept, r_value, p_value, std_err = linregress(df_jul_global_temp['year'], df_jul_global_temp['anomaly'])
slope_aug, intercept, r_value, p_value, std_err = linregress(df_aug_global_temp['year'], df_aug_global_temp['anomaly'])
slope_sep, intercept, r_value, p_value, std_err = linregress(df_sep_global_temp['year'], df_sep_global_temp['anomaly'])
slope_oct, intercept, r_value, p_value, std_err = linregress(df_oct_global_temp['year'], df_oct_global_temp['anomaly'])
slope_nov, intercept, r_value, p_value, std_err = linregress(df_nov_global_temp['year'], df_nov_global_temp['anomaly'])
slope_dec, intercept, r_value, p_value, std_err = linregress(df_dec_global_temp['year'], df_dec_global_temp['anomaly'])

round_slope_jan = round(slope_jan, 4)
round_slope_feb = round(slope_feb, 4)
round_slope_mar = round(slope_mar, 4)
round_slope_apr = round(slope_apr, 4)
round_slope_may = round(slope_may, 4)
round_slope_jun = round(slope_jun, 4)

round_slope_jul = round(slope_jul, 4)
round_slope_aug = round(slope_aug, 4)
round_slope_sep = round(slope_sep, 4)
round_slope_oct = round(slope_oct, 4)
round_slope_nov = round(slope_nov, 4)
round_slope_dec = round(slope_dec, 4)

slope_months= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec (excl. 2023)']
slope_calculations = np.array([round_slope_jan, round_slope_feb, round_slope_mar, round_slope_apr, round_slope_may, round_slope_jun, round_slope_jul, round_slope_aug, round_slope_sep, round_slope_oct, round_slope_nov, round_slope_dec])

df_regression_slopes_for_anomalies_by_year = pd.DataFrame({'month':slope_months, 'slope':slope_calculations})

print(f'The slope of the regression line for the global temperature anomaly for each month for the period is as follows:')
print()
print(df_regression_slopes_for_anomalies_by_year)

Whatever the causes might be, it appears that over the past six years, the global temperature anomaly during the latter months of the year has been increasing at a greater rate than the earlier months of the year; could it be possible this is the result of a very recent (past six months) acceleration of the rate of warming which is dragging up the values for temperature anomaly for these months for 2023, which in turn causes the slope of the regression line to be steeper? When I first calculated these figures, I was unaware that the recent past has been an El Niño period, and as I describe shortly, this is associated with temporarily elevated figures for global temperature.

I will print plots (without saving) of the global temperature anomaly for September (the month with steepest slope), October and November to investigate:

In [ ]:
plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) September (2018 to 2023)')
plt.xlabel('September - by year')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='year', y='anomaly', data=df_sep_global_temp, order=1, color = 'green')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) October (2018 to 2023)')
plt.xlabel('October - by year')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='year', y='anomaly', data=df_oct_global_temp, order=1, color = 'green')
plt.show()

plt.figure(figsize= (5,5))
plt.title('Global temperature anomaly (in °C) November (2018 to 2023)')
plt.xlabel('November - by year')
plt.ylabel('Temperature Anomaly (°C)')
sns.regplot(x='year', y='anomaly', data=df_nov_global_temp, order=1, color = 'green')
plt.show()

Here was can see that for September, October and November, the most recent figure for temperature anomaly is significantly higher than the equivilant figures for the five years prior to that; this causes the slop line to pivot upwards and become steeper.

*Percentage increases/decreases in temperature anomalies over 2023:*

I now want to calculate the percentage increase/decrease of the global temperature anomaly for each month of 2023 compared with the month prior to it (with the exception of December):

In [ ]:
# here I am using the pct_change() method to calculate the percentage changes and multiplying
# the result by 100 to turn it into a percentage
df_2023_global_temp_percentages = df_2023_global_temp['anomaly'].pct_change() * 100

months_2023 = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']

# create a new dataframe showing Month (of 2023) and the percentage increase in temperature anomaly over the previous month
df_monthly_percentage_increases_temp_anomaly_2023 = pd.DataFrame({'month':months_2023, 'percentage increase':df_2023_global_temp_percentages})

print(f'The percentage increase in the temperature anomaly for each month in 2023 over the previous month is as follows:')
print()
print(df_monthly_percentage_increases_temp_anomaly_2023)

We can see that from June to November 2023, the temperature anomaly increased in percentage terms for four consecutive months in a row from June to September, which represents an acceleration in warming over those four months. There was then a fall of 4.89% in October from the high point of September, followed by a rise again of 5.88% in November over Octobers figure. 

A report on the BBC website dated 9th January 2024 describes how 2023 was the warmest year on record and outlines the many records which were broken. In particular, the second half of the year seems to have had particularly extreme temperature, with the article describing an "almost unbroken streak of daily records in the second half of 2023", and more than 200 days saw the daily record be broken.

<img src="./images/2023-temp-record-streak.png" alt="daily temperature record streak">


The global temperature was 1.48 °C higher on average than the long-term average before humans started burning lots of fossil fuels. 

The article describes how, in addition to anthropogenic global warming, the high temperature was due to the El Niño effect; this occurs when the winds which normally blow warm water from east to west start blowing in the other direction, sending warm water eastwards instead. The El Niño effect causes average temperature worldwide to increase. This article (https://www.bbc.co.uk/news/science-environment-64192508 explaining the El Niño effect also outlines how 2020, 2021 and 2022 where La Niña years instead, where warm waters move westwards rather than eastwards, which leads to opposite effects to El Niño events. El Niño and La Niña episodes typically occur every two to seven years, and usually last nine to 12 months, with La Niña episodes being less common than El Niño. I didn't realise when I investigated the temperatures earlier that 2020 to 2022 was impacted by the La Niña effect, but it agrees with my findings of lower temperatures in 2021 and 2022 than in 2019, 2020 and 2023.

Since the high temperatures of 2023 were linked to the El Niño phenomenon, it remains to be seen whether the higher figures for the global temperature anomaly for 2023 relative to earlier years will persist on into 2024 and whether the warming will accelerate. El Niño is expected to last until at least April 2024.

### Summary ###

That concludes my investigation into climate data from the Paleolithic to the present day. I found this a challenging project but an engaging one, and I was fortunate to have a pre-existing interest in the subject matter of the project before I began. I had some familiarity with the science but I have learned a lot more over the course of the project and have enjoyed doing so.

With all the datasets I used, I tried to cleanse the data as best I could to give myself the best chance of being able to discern the relationships existing behind the data. Crucially, I standardised the measurement of time in each dataset by including a column called 'Years' which represented time in terms of "years ago".

Throughout the project, I examined datasets for the history of atmospheric carbon dioxide concentrations, methane concentrations and Anatarctic temperature anomalies, and the relationships existing between each of these. I examined from timescales from 820,000 years ago until the present day. 

I carried out correlation analyses between recent data for CO2, CH4 and the global temperature anomaly and lead-lag analyses between CO2 and the temperature anomaly, and CO2 and CH4. For CO2 and the temperature anomaly, my correlations implied a slightly stronger correlation between future CO2 and present temperature, which is difficult to interpret. More intuitively, for CO2 and CH4, I found that there is a slight leading relationship between CO2 and CH4, where past CO2 is slightly more strongly associated with present CH4 levels.

I then analysed the relationships between temperature and rainfall anomalies for Mullingar weather station (in Ireland), and the Mullingar temperature anomaly against the Global temperature anomaly. I chose to analyse a single weather station rather than a composite Irish dataset, as the data was easy to find, accurate and complete going back to 1950, and I was satisfied with only testing that far back into the past.

Next, I fused together all of the datasets I had worked on until that point into CSV and json files (each called 'composite climate data') and exported them to my subfolder in this directory called 'climate data'; these datasets could prove useful in future to any person looking to carry out similar analyses on climate data and who wants to have multiple datasets in the one place.

I then made predictions for the Global temperature anomaly for the next few decades using a simple linear regression model based on the data from the NOAA for that anomaly from 1950 to 2022. I compared this with the published temperature prediction models used by the Intergovernmental Panel on Climate Chang; I found that the results of my linear regression model are compare to the more optimistic outcomes envisaged by the IPCC, suggesting that a linear model may not be capture the shocking extent to which the climate is warming up. Indeed, when I investigated for an acceleration in the rate of warming over 2023, I found some evidence in support of that possibilty.

The data leaves us in no doubt as to the shocking impact which human activities have so far had on the climate; what is more shocking, however, is the thought of how bad things are going to be in the coming decades even in a best case scenario. Like most people, I hope that this worst effects can be mitigated over time but even achieving some degree of mitigation will require huge permanent changes and sacrifice by all, and, as we saw during the pandemic, people find it hard to accept reduced living standards and freedom once they have become accustomed to it, even when they are told that their forgoing it is for the public good. There is also the issue of some countries not taking the issue as seriously as others, or simply not being able to afford to look at the issue in the same way as Westerners can; in much poorer countries, the focus is on immediate survival and getting through the day, and if that requires burning whatever fuel they can or eating whatever foods they feel they need to nourish themselves and their families, they will not be subordinating their short-term needs for the long-term sake of the climate. On the other hand, efforts to tackle the problem need to be successful, because the effects of climate change will be disasterous on a global scale if we don't.

### References ###

*Introduction*

1.	(Rebecca Lindsey) 12th May 2023.*Climate Change: Atmospheric Carbon Dioxide* Climate.gov. Retrieved 1st January 2024 from https://www.climate.gov/news-features/understanding-climate/climate-change-atmospheric-carbon-dioxide 

2.	(Ben Henley and Nerilie Abram) 6th June 2017. *The three-minute story of 800,000 years of climate change with a sting in the tail* Theconversation.com. Retrieved 1st January 2024 from https://theconversation.com/the-three-minute-story-of-800-000-years-of-climate-change-with-a-sting-in-the-tail-73368  

*Data Cleansing*

3.	(Tans, P and Thoning, K) December 2020. *How we measure background CO2 levels on Mauna Loa.* Gml.noaa.gov. Retrieved 1st January 2024 from https://gml.noaa.gov/ccgg/about/co2_measurements.html 

4.	(Tans, P and Keeling, R) n/d. *Trends in Atmospheric Carbon Dioxide* Gml.noaa.gov. Retrieved 1st January 2024 from https://gml.noaa.gov/ccgg/trends/data.html 

5.	(Mulvaney, R) 20th September 2004. *How are past temperatures determined from an ice core?* Scientificamerican.com. Retrieved 1st January 2024 from https://www.scientificamerican.com/article/how-are-past-temperatures/ 

6.	(Lan, X., Thoning, K.W and Dlugokencky, E.J) n/d. *Trends in Atmospheric Methane* gml.noaa.gov. Retrieved 1st January 2024 from https://gml.noaa.gov/ccgg/trends_ch4/ 

7.	(Many authors) 26th November 2007. *EPICA Dome C Ice Core 800KYr Deuterium Data and Temperature Estimates* ncei.noaa.gov. Retrieved 1st January 2024 from https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt

8.	(author/s unknown) n/d. *Jean Jouzel* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Jean_Jouzel

9.	(many authors) n/d. *Global Time Series* ncei.noaa.gov. Retrieved 1st January 2024 from https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/global/time-series/globe/land_ocean/1/11/1850-2023 

10.	(author/s unknown) n/d. *seaborn.scatterplot* Seaborn.pydata.org. Retrieved 1st January 2024 from https://seaborn.pydata.org/generated/seaborn.scatterplot.html

11.	(author/s unknown) n/d. *How to create new columns derived from existing columns* Pandas.pydata.org. Retrieved 1st January 2024 from https://pandas.pydata.org/docs/getting_started/intro_tutorials/05_add_columns.html 

12.	(Schork, J) n/d. *Reverse pandas DataFrame in Python (3 Examples)* Statisticsglobe.com. Retrieved 1st January 2024 from https://statisticsglobe.com/reverse-pandas-dataframe-in-python#example-1-reverse-ordering-of-dataframe-rows 

13.	(author/s unknown) n/d. *pandas.DataFrame.astype* Pandas.pydata.org. Retrieved 1st January 2024 from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html

14.	(author/s unknown) n/d. *Returning a view versus a copy* Pandas.pydata.org. Retrieved 1st January 2024 from https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy 

15.	(author/s unknown) n/d *pandas.DataFrame.combine_first* Pandas.pydata.org. Retrieved 1st January 2024 from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html 

16.	(author/s unknown) n/d *pandas.DataFrame.drop* Pandas.pydata.org. Retrieved 1st January 2024 from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html


*CO2 versus Temperature Anomaly*

17.	(author/s unknown) n/d. *pandas.DataFrame.shift* Pandas.pydata.org. Retrieved 10th January 2024 from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html

18.	(Bajwa, A) n/d. *How to create lags and leads of a column in a pandas DataFrame* Educative.io. Retrieved 10th January 2024 from https://www.educative.io/answers/how-to-create-lags-and-leads-of-a-column-in-a-pandas-dataframe


*CH4 versus Temperature Anomaly*

19.	(author/s unknown) n/d. *Atmospheric methane* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Atmospheric_methane 

20.	(author/s unknown) n/d. *Dichlorodifluoromethane* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Dichlorodifluoromethane 

21.	(author/s unknown) n/d. *Global warming potential* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Global_warming_potential 

22.	(McGrath, M) 22nd May 2019. *Ozone layer: Banned CFCs traced to China say scientists* Bbc.com. Retrieved 1st January 2024 from https://www.bbc.com/news/science-environment-48353341 

23.	(author/s unknown) n/d. *Methane and climate change* iea.org. Retrieved 1st January 2024 from https://www.iea.org/reports/global-methane-tracker-2022/methane-and-climate-change 

24.	(author/s unknown) July 2022. *Climate Change Indicators: Atmospheric Concentrations of Greenhouse Gases* epa.gov. Retrieved 1st January 2024 from https://www.epa.gov/climate-indicators/climate-change-indicators-atmospheric-concentrations-greenhouse-gases 

25.	(author/s unknown) n/d. *Eemian* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Eemian 

26.	(author/s unknown) n/d. *Little Ice Age* Wikipedia.org. Retrieved 1st January 2024 from https://en.wikipedia.org/wiki/Little_Ice_Age 

27.	(author/s uknown) n/d. *Global temperature record* Temperaturerecord.org. Retrieved 1st January 2024 from https://www.temperaturerecord.org/ 

*CH4 versus CO2*

28.	(Mearns, E) 12th December 2014. *The Vostok Ice Core: Temperature, CO2 and CH4* Eaunmearns.com. Retrieved 1st January 2024 from https://euanmearns.com/the-vostok-ice-core-temperature-co2-and-ch4/     


*Climate Change signals in the Irish context*

29.	(author/s unknown) n/d *Hacks* Markdownguide.org. Retrieved 9th January 2024 from https://www.markdownguide.org/hacks/

30.	(Murphy, C) 20th October 2023. *How we detected human induced climate change in Irish weather records* Rte.ie. Retrieved 1st January 2024 from https://www.rte.ie/brainstorm/2023/1020/1412054-human-induced-climate-change-irish-weather/

31.	(Multiple authors) December 2023. *The emergence of a climate change signal in long-term Irish meteorological observations* Sciencedirect.com. Retrieved 1st January 2024 from https://www.sciencedirect.com/science/article/pii/S2212094723000610 

32.	(Mateus, C and Potito, A) June 2022. *Long-term trends in daily extreme air temperature indices in Ireland from 1885 to 2018* sciencedirect.com. Retrieved 1st January 2024 from https://www.sciencedirect.com/science/article/pii/S2212094722000469

33.	(author/s unknown) n/d. *Met Éireann 1961-1990 Climate averages* Data.gov.ie. Retrieved 5th January 2024 from https://data.gov.ie/dataset/met-eireann-1961-1990-climate-averages

34.	(author/s unknown) 22nd April 2023. *Where is Mullingar Ireland? Map of Mullingar* Whereismap.net. Retrieved 6th January 2024 from https://whereismap.net/wp-content/uploads/2022/04/Where-is-Mullingar-Ireland-800x445.jpg 


*Fusion of datasets*

35.	(Olumide, S) 24th March 2023. *Dataframe to CSV – How to Save Pandas Dataframes by Exporting* Freecodecamp.org. Retrieved 1st January 2024 from https://www.freecodecamp.org/news/dataframe-to-csv-how-to-save-pandas-dataframes-by-exporting/

36.	(BPRASAD26) (n/d) *How to Export a Pandas Dataframe to a JSON File?* Lifewithdata.com. Retrieved 1st January 2024 from https://lifewithdata.com/2022/07/11/how-to-export-a-pandas-dataframe-to-a-json-file/

37.	(McGinley, B – lecturer at ATU) 10th November 2023. *SKLearn – Regression* https://vlegalwaymayo.atu. Retrieved 5th January 2024 from https://vlegalwaymayo.atu.ie/course/view.php?id=8876 


*Prediction of Global Temperature Anomaly*

38.	(author/s unknown) n/d. *sklearn.linear_model.LinearRegression* scikit-learn.org. Retrieved 5th January 2024 from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

39.	(many authors) 2021. *Summary for Policymakers* ipcc.ch. Retrieved 9th January 2024 from https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_SPM.pdf.

40.	Climateknowledgeportal.worldbank.org. Retrieved 9th January 2024 from https://climateknowledgeportal.worldbank.org/sites/default/files/inline-images/SSPs.png (source of image, originally from ipcc report)

41.	(author/s unknown) n/d *Annual carbon dioxide (CO₂) emissions worldwide from 1940 to 2023* statista.com. Retrieved 9th January from https://www.statista.com/statistics/276629/global-co2-emissions


*Accelerating increases of Temperature*

42.	(author/s unknown) n/d. *Pandas DataFrame pct_change() Method* W3schools.com. Retrieved 9th January 2024 from https://www.w3schools.com/python/pandas/ref_df_pct_change.asp

43.	(Poynting, M and Rivault, E) 9th January 2024. *2023 confirmed as world's hottest year on record* Bbc.com. Retrieved 9th January 2024 from https://www.bbc.com/news/science-environment-67861954

44.	(Stallard, E and Poytning, M) 8th June 2023. *What are El Niño and La Niña, and how do they change the weather?* Bbc.com. Retrieved 9th January 2024 from https://www.bbc.com/news/science-environment-64192508

45.	(author/s unknown) 8th November 2023. *El Niño expected to last at least until April 2024* Wmo.int. Retrieved 9th January 2024 from https://wmo.int/news/media-centre/el-nino-expected-last-least-until-april-2024


*General*

46. ('Gisgar3') 17th February 2022. *Github pages' images not working* Github.com. Retrieved 10th January 2024 from https://github.com/orgs/community/discussions/23086.